# Setup and Function Definition

In [1]:
import pandas as pd
from urllib.request import Request, urlopen
import matplotlib.pyplot as plt
import numpy as np
import requests

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

def event_during_shift(event_time, event_period, shift_start, shift_end, shift_period):
        
    event_min = event_time[0:2]
    event_sec = event_time[3:5]
    shift_start_min = shift_start[0:2]
    shift_start_sec = shift_start[3:5]
    shift_end_min = shift_end[0:2]
    shift_end_sec = shift_end[3:5]

    # shift start and end and event all in same minute (ex. event 01:28, 01:20 - 01:59)
    if ((event_min >= shift_start_min) and (event_sec >= shift_start_sec) and
        (event_min <= shift_end_min) and (event_sec < shift_end_sec)):
        return True

    # shift start and event in same minute, shift end later (ex. event 01:28, 01:20 - 02:04)
    elif ((event_min >= shift_start_min) and (event_sec >= shift_start_sec) and (event_min < shift_end_min)):
        return True

    # shift end and event in same minute, shift start earlier (ex. event 1:28, 00:56 - 1:59)
    elif ((event_min > shift_start_min) and (event_min <= shift_end_min) and (event_sec < shift_end_sec)):
        return True

    # shift start and end and event all in separate minutes (ex. event 03:29, 02:58 - 04:05)
    elif ((event_min > shift_start_min) and (event_min < shift_end_min)):
        return True

    # OT goal exception
    elif (event_period == 4 and event_min == shift_end_min and event_sec == shift_end_sec):
        return True

    # shootout exception
    elif event_period == 5:
        return False

    else:
        return False

# 2018-2019 Regular Season

In [ ]:
# create lists to store events for the entire season, to be converted to Dataframes

# events_df
event_id = []
game_id = []
player1_id = []
player1_name = []
player1_link = []
player1_type = []
player2_id = []
player2_name = []
player2_link = []
player2_type = []
primaryAssister_id = []
primaryAssister_name = []
primaryAssister_link = []
primaryAssister_type = []
primaryAssister_total = []
secondaryAssister_id = []
secondaryAssister_name = []
secondaryAssister_link = []
secondaryAssister_type = []
secondaryAssister_total = []
penaltyServer_id = []
penaltyServer_name = []
penaltyServer_link = []
penaltyServer_type = []
strength = []
gwg = []
emptyNet = []
scorer_total = []
event = []
eventCode = []
eventType = []
eventDescription = []
secondaryType = []
penaltySeverity = []
penaltyMinutes = []
eventIdx = []
period = []
periodType = []
periodTime = []
dateTime = []
awayGoals = []
homeGoals = []
xCoord = []
yCoord = []
team1_id = []
team1_name = []
team1_link = []
team1_tricode = []
away_p1 = []
away_p2 = []
away_p3 = []
away_p4 = []
away_p5 = []
away_p6 = []
home_p1 = []
home_p2 = []
home_p3 = []
home_p4 = []
home_p5 = []
home_p6 = []

# 2018-2019 regular season had 1271 total games
root = '201802'

for game in range(1, 1272): #1,1272
    
    if game<10:
        code = root+'000'+str(game)
    elif game<100:
        code = root+'00'+str(game)
    elif game<1000:
        code = root+'0'+str(game)
    else:
        code = root+str(game)
        
    print(code)
    
    # access game data
    game_data_url = 'https://statsapi.web.nhl.com/api/v1/game/'+str(code)+'/feed/live'
    req = Request(game_data_url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    page_json = json.loads(webpage.decode('utf-8')) ##webpage
    game_data = page_json['gameData']
    events = page_json['liveData']['plays']['allPlays']
    
    # get away and home team abbreviations
    away_abbrev = game_data['teams']['away']['abbreviation']
    home_abbrev = game_data['teams']['home']['abbreviation']
    
    # access corresponding shift data and build dataframe
    shift_data_url = 'https://api.nhle.com/stats/rest/en/shiftcharts?cayenneExp=gameId='+str(code)
    req = Request(shift_data_url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    shift_json = json.loads(webpage.decode('utf-8')) ##webpage
    
    shift_player_id = []
    shift_player_name = []
    shift_player_team = []
    shift_period = []
    shift_start = []
    shift_end = []
    shifts = shift_json['data']
    
    for i in range(len(shifts)):
        shift_player_id.append(shifts[i]['playerId'])
        player_first = shifts[i]['firstName']
        player_last = shifts[i]['lastName']
        shift_player_name.append(player_first + ' ' + player_last)
        shift_player_team.append(shifts[i]['teamAbbrev'])
        shift_period.append(shifts[i]['period'])
        shift_start.append(shifts[i]['startTime'])
        shift_end.append(shifts[i]['endTime'])
        
    shift_df = pd.DataFrame({
        'playerID': shift_player_id,
        'player name': shift_player_name,
        'player team': shift_player_team,
        'period': shift_period,
        'shift start': shift_start,
        'shift end': shift_end
    })
    
    # iterate through all events and store as a row
    for i in range(len(events)):
    
        # add game_id to every row
        game_id.append(game_data['game']['pk'])

        # these keys exist for every event, whether hockey play or not
        event_result = events[i]['result']
        event_about = events[i]['about']
        
        eventIdx.append(event_about['eventIdx'])
        event_id.append(str(game_data['game']['pk']) + '-' + str(event_about['eventIdx'])) # primary key for nhl_events
        event.append(event_result['event'])
        eventCode.append(event_result['eventCode'])
        eventType.append(event_result['eventTypeId'])
        eventDescription.append(event_result['description'])
        period.append(event_about['period'])
        periodType.append(event_about['periodType'])
        periodTime.append(event_about['periodTime'])
        dateTime.append(event_about['dateTime'])
        awayGoals.append(event_about['goals']['away'])
        homeGoals.append(event_about['goals']['home'])
    
        # event is a hockey event (Shot, Goal, Hit, etc.)
        if 'players' in events[i]:

            player_info = events[i]['players']

            # event type is a Giveaway, Takeaway, or Missed Shot
            if len(player_info) == 1:
                player1_id.append(player_info[0]['player']['id'])
                player1_name.append(player_info[0]['player']['fullName'])
                player1_link.append(player_info[0]['player']['link'])
                player1_type.append(player_info[0]['playerType'])
                player2_id.append('NULL')
                player2_name.append('NULL')
                player2_link.append('NULL')
                player2_type.append('NULL')
                primaryAssister_id.append('NULL')
                primaryAssister_name.append('NULL')
                primaryAssister_link.append('NULL')
                primaryAssister_type.append('NULL')
                primaryAssister_total.append('NULL')
                secondaryAssister_id.append('NULL')
                secondaryAssister_name.append('NULL')
                secondaryAssister_link.append('NULL')
                secondaryAssister_type.append('NULL')
                secondaryAssister_total.append('NULL')
                penaltyServer_id.append('NULL')
                penaltyServer_name.append('NULL')
                penaltyServer_link.append('NULL')
                penaltyServer_type.append('NULL')
            # event type is not a Goal, Giveaway, Takeaway, or Missed Shot or it's a goal with no assists
            elif len(player_info) == 2:
                player1_id.append(player_info[0]['player']['id'])
                player1_name.append(player_info[0]['player']['fullName'])
                player1_link.append(player_info[0]['player']['link'])
                player1_type.append(player_info[0]['playerType'])
                player2_id.append(player_info[1]['player']['id'])
                player2_name.append(player_info[1]['player']['fullName'])
                player2_link.append(player_info[1]['player']['link'])
                player2_type.append(player_info[1]['playerType'])   
                primaryAssister_id.append('NULL')
                primaryAssister_name.append('NULL')
                primaryAssister_link.append('NULL')
                primaryAssister_type.append('NULL')
                primaryAssister_total.append('NULL')
                secondaryAssister_id.append('NULL')
                secondaryAssister_name.append('NULL')
                secondaryAssister_link.append('NULL')
                secondaryAssister_type.append('NULL')
                secondaryAssister_total.append('NULL')
                penaltyServer_id.append('NULL')
                penaltyServer_name.append('NULL')
                penaltyServer_link.append('NULL')
                penaltyServer_type.append('NULL')
            # event type is a Goal with one assist or a Penalty taken and served by different players
            elif len(player_info) == 3:
                player1_id.append(player_info[0]['player']['id'])
                player1_name.append(player_info[0]['player']['fullName'])
                player1_link.append(player_info[0]['player']['link'])
                player1_type.append(player_info[0]['playerType'])
                # event type is Goal
                if event_result['eventTypeId'] == 'GOAL':
                    primaryAssister_id.append(player_info[1]['player']['id'])
                    primaryAssister_name.append(player_info[1]['player']['fullName'])
                    primaryAssister_link.append(player_info[1]['player']['link'])
                    primaryAssister_type.append(player_info[1]['playerType'])
                    primaryAssister_total.append(player_info[1]['seasonTotal'])
                    player2_id.append(player_info[2]['player']['id'])
                    player2_name.append(player_info[2]['player']['fullName'])
                    player2_link.append(player_info[2]['player']['link'])
                    player2_type.append(player_info[2]['playerType'])
                    penaltyServer_id.append('NULL')
                    penaltyServer_name.append('NULL')
                    penaltyServer_link.append('NULL')
                    penaltyServer_type.append('NULL')
                # event type is Penalty
                elif event_result['eventTypeId'] == 'PENALTY':
                    player2_id.append(player_info[1]['player']['id'])
                    player2_name.append(player_info[1]['player']['fullName'])
                    player2_link.append(player_info[1]['player']['link'])
                    player2_type.append(player_info[1]['playerType'])
                    penaltyServer_id.append(player_info[2]['player']['id'])
                    penaltyServer_name.append(player_info[2]['player']['fullName'])
                    penaltyServer_link.append(player_info[2]['player']['link'])
                    penaltyServer_type.append(player_info[2]['playerType'])
                    primaryAssister_id.append('NULL')
                    primaryAssister_name.append('NULL')
                    primaryAssister_link.append('NULL')
                    primaryAssister_type.append('NULL')
                    primaryAssister_total.append('NULL')
                else:
                    print(game_data['game']['pk'])   
                secondaryAssister_id.append('NULL')
                secondaryAssister_name.append('NULL')
                secondaryAssister_link.append('NULL')
                secondaryAssister_type.append('NULL')
                secondaryAssister_total.append('NULL')
            # event type is a Goal with two assists
            elif len(player_info) == 4:
                player1_id.append(player_info[0]['player']['id'])
                player1_name.append(player_info[0]['player']['fullName'])
                player1_link.append(player_info[0]['player']['link'])
                player1_type.append(player_info[0]['playerType'])
                player2_id.append(player_info[3]['player']['id'])
                player2_name.append(player_info[3]['player']['fullName'])
                player2_link.append(player_info[3]['player']['link'])
                player2_type.append(player_info[3]['playerType']) 
                primaryAssister_id.append(player_info[1]['player']['id'])
                primaryAssister_name.append(player_info[1]['player']['fullName'])
                primaryAssister_link.append(player_info[1]['player']['link'])
                primaryAssister_type.append(player_info[1]['playerType'])
                primaryAssister_total.append(player_info[1]['seasonTotal'])
                secondaryAssister_id.append(player_info[2]['player']['id'])
                secondaryAssister_name.append(player_info[2]['player']['fullName'])
                secondaryAssister_link.append(player_info[2]['player']['link'])
                secondaryAssister_type.append(player_info[2]['playerType'])
                secondaryAssister_total.append(player_info[2]['seasonTotal'])
                penaltyServer_id.append('NULL')
                penaltyServer_name.append('NULL')
                penaltyServer_link.append('NULL')
                penaltyServer_type.append('NULL')

            # event is a Goal, Shot, or Penalty
            if 'secondaryType' in event_result:
                secondaryType.append(event_result['secondaryType'])
            else:
                secondaryType.append('NULL')

            # event is a Penalty
            if event_result['eventTypeId'] == 'PENALTY':
                penaltySeverity.append(event_result['penaltySeverity'])
                penaltyMinutes.append(event_result['penaltyMinutes'])
            else:
                penaltySeverity.append('NULL')
                penaltyMinutes.append('NULL')

            # event is a Goal
            if event_result['eventTypeId'] == 'GOAL':
                strength.append(event_result['strength']['code'])
                gwg.append(event_result['gameWinningGoal'])
                # not sure why but some goal events are missing emptyNet key
                if 'emptyNet' in event_result:
                    emptyNet.append(event_result['emptyNet'])
                else:
                    emptyNet.append('NULL')
                scorer_total.append(player_info[0]['seasonTotal'])
            else:
                strength.append('NULL')
                gwg.append('NULL')
                emptyNet.append('NULL')
                scorer_total.append('NULL')


            # regardless of event type
            team1_id.append(events[i]['team']['id'])
            team1_name.append(events[i]['team']['name'])
            team1_link.append(events[i]['team']['link'])
            team1_tricode.append(events[i]['team']['triCode'])
            # some events are missing coordinates, no idea why
            if events[i]['coordinates']:
                xCoord.append(events[i]['coordinates']['x'])
                yCoord.append(events[i]['coordinates']['y'])
            else:
                xCoord.append('Missing')
                yCoord.append('Missing')

            # players on ice for each event
            away_players_on_ice = []
            home_players_on_ice = []
            
            for shift in range(len(shift_df)):
                if event_about['period'] == shift_df['period'][shift]:
                    if event_during_shift(event_about['periodTime'], event_about['period'], shift_df['shift start'][shift],
                                            shift_df['shift end'][shift], shift_df['period'][shift]) == True:
                        if shift_df['player team'][shift] == away_abbrev:
                            away_players_on_ice.append(shift_df['player name'][shift])
                        elif shift_df['player team'][shift] == home_abbrev:
                            home_players_on_ice.append(shift_df['player name'][shift])

            if len(away_players_on_ice)>=1:
                away_p1.append(away_players_on_ice[0])
            else:
                away_p1.append('NULL')
            if len(away_players_on_ice)>=2:
                away_p2.append(away_players_on_ice[1])
            else:
                away_p2.append('NULL')
            if len(away_players_on_ice)>=3:
                away_p3.append(away_players_on_ice[2])
            else:
                away_p3.append('NULL')
            if len(away_players_on_ice)>=4:
                away_p4.append(away_players_on_ice[3])
            else:
                away_p4.append('NULL')
            if len(away_players_on_ice)>=5:
                away_p5.append(away_players_on_ice[4])
            else:
                away_p5.append('NULL')
            if len(away_players_on_ice)>=6:
                away_p6.append(away_players_on_ice[5])
            else:
                away_p6.append('NULL')

            if len(home_players_on_ice)>=1:
                home_p1.append(home_players_on_ice[0])
            else:
                home_p1.append('NULL')
            if len(home_players_on_ice)>=2:
                home_p2.append(home_players_on_ice[1])
            else:
                home_p2.append('NULL')
            if len(home_players_on_ice)>=3:
                home_p3.append(home_players_on_ice[2])
            else:
                home_p3.append('NULL')
            if len(home_players_on_ice)>=4:
                home_p4.append(home_players_on_ice[3])
            else:
                home_p4.append('NULL')
            if len(home_players_on_ice)>=5:
                home_p5.append(home_players_on_ice[4])
            else:
                home_p5.append('NULL')
            if len(home_players_on_ice)>=6:
                home_p6.append(home_players_on_ice[5])
            else:
                home_p6.append('NULL')


        # event is a non-hockey event (Game start/end, Period start/end, Stoppage, etc.)
        else:
            player1_id.append('NULL')
            player1_name.append('NULL')
            player1_link.append('NULL')
            player1_type.append('NULL')
            player2_id.append('NULL')
            player2_name.append('NULL')
            player2_link.append('NULL')
            player2_type.append('NULL')
            primaryAssister_id.append('NULL')
            primaryAssister_name.append('NULL')
            primaryAssister_link.append('NULL')
            primaryAssister_type.append('NULL')
            primaryAssister_total.append('NULL')
            secondaryAssister_id.append('NULL')
            secondaryAssister_name.append('NULL')
            secondaryAssister_link.append('NULL')
            secondaryAssister_type.append('NULL')
            secondaryAssister_total.append('NULL')
            secondaryType.append('N/A')
            penaltySeverity.append('N/A')
            penaltyMinutes.append('N/A')
            penaltyServer_id.append('NULL')
            penaltyServer_name.append('NULL')
            penaltyServer_link.append('NULL')
            penaltyServer_type.append('NULL')
            strength.append('NULL')
            gwg.append('NULL')
            emptyNet.append('NULL')
            scorer_total.append('NULL')
            xCoord.append('NULL')
            yCoord.append('NULL')
            team1_id.append('NULL')
            team1_name.append('NULL')
            team1_link.append('NULL')
            team1_tricode.append('NULL')
            away_p1.append('NULL')
            away_p2.append('NULL')
            away_p3.append('NULL')
            away_p4.append('NULL')
            away_p5.append('NULL')
            away_p6.append('NULL')
            home_p1.append('NULL')
            home_p2.append('NULL')
            home_p3.append('NULL')
            home_p4.append('NULL')
            home_p5.append('NULL')
            home_p6.append('NULL')

# convert resulting season-long lists into Dataframe
reg20182019_events_df = pd.DataFrame({
    'event_id': event_id,
    'game_id': game_id,
    'player1_id': player1_id,
    'player1_name': player1_name,
    'player1_link': player1_link,
    'player1_type': player1_type,
    'player2_id': player2_id,
    'player2_name': player2_name,
    'player2_link': player2_link,
    'player2_type': player2_type,
    'event': event,
    'eventCode': eventCode,
    'eventType': eventType,
    'eventDescription': eventDescription,
    'secondaryType': secondaryType,
    'xCoord': xCoord,
    'yCoord': yCoord,
    'primaryAssister_id': primaryAssister_id,
    'primaryAssister_name': primaryAssister_name,
    'primaryAssister_link': primaryAssister_link,
    'primaryAssister_type': primaryAssister_type,
    'primaryAssister_total': primaryAssister_total,
    'secondaryAssister_id': secondaryAssister_id,
    'secondaryAssister_name': secondaryAssister_name,
    'secondaryAssister_link': secondaryAssister_link,
    'secondaryAssister_type': secondaryAssister_type,
    'secondaryAssister_total': secondaryAssister_total,
    'penaltyServer_id': penaltyServer_id,
    'penaltyServer_name': penaltyServer_name,
    'penaltyServer_link': penaltyServer_link,
    'penaltyServer_type': penaltyServer_type,
    'strength': strength,
    'gwg': gwg,
    'emptyNet': emptyNet,
    'scorer_total': scorer_total,
    'penaltySeverity': penaltySeverity,
    'penaltyMinutes': penaltyMinutes,
    'eventIdx': eventIdx,
    'period': period,
    'periodType': periodType,
    'periodTime': periodTime,
    'dateTime': dateTime,
    'awayGoals': awayGoals,
    'homeGoals': homeGoals,
    'team1_id': team1_id,
    'team1_name': team1_name,
    'team1_link': team1_link,
    'team1_tricode': team1_tricode,
    'away p1': away_p1,
    'away p2': away_p2,
    'away p3': away_p3,
    'away p4': away_p4,
    'away p5': away_p5,
    'away p6': away_p6,
    'home p1': home_p1,
    'home p2': home_p2,
    'home p3': home_p3,
    'home p4': home_p4,
    'home p5': home_p5,
    'home p6': home_p6
})

# 2019-2020 Regular Season

In [ ]:
# create lists to store events for the entire season, to be converted to Dataframes

# events_df
event_id = []
game_id = []
player1_id = []
player1_name = []
player1_link = []
player1_type = []
player2_id = []
player2_name = []
player2_link = []
player2_type = []
primaryAssister_id = []
primaryAssister_name = []
primaryAssister_link = []
primaryAssister_type = []
primaryAssister_total = []
secondaryAssister_id = []
secondaryAssister_name = []
secondaryAssister_link = []
secondaryAssister_type = []
secondaryAssister_total = []
penaltyServer_id = []
penaltyServer_name = []
penaltyServer_link = []
penaltyServer_type = []
strength = []
gwg = []
emptyNet = []
scorer_total = []
event = []
eventCode = []
eventType = []
eventDescription = []
secondaryType = []
penaltySeverity = []
penaltyMinutes = []
eventIdx = []
period = []
periodType = []
periodTime = []
dateTime = []
awayGoals = []
homeGoals = []
xCoord = []
yCoord = []
team1_id = []
team1_name = []
team1_link = []
team1_tricode = []
away_p1 = []
away_p2 = []
away_p3 = []
away_p4 = []
away_p5 = []
away_p6 = []
home_p1 = []
home_p2 = []
home_p3 = []
home_p4 = []
home_p5 = []
home_p6 = []

# 2019-2020 regular season had 1082 total games
root = '201902'

for game in range(1, 10): #1,1083
    
    if game<10:
        code = root+'000'+str(game)
    elif game<100:
        code = root+'00'+str(game)
    elif game<1000:
        code = root+'0'+str(game)
    else:
        code = root+str(game)
    
    # access game data
    game_data_url = 'https://statsapi.web.nhl.com/api/v1/game/'+str(code)+'/feed/live'
    req = Request(game_data_url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    page_json = json.loads(webpage.decode('utf-8')) ##webpage
    game_data = page_json['gameData']
    events = page_json['liveData']['plays']['allPlays']
    
    # get away and home team abbreviations
    away_abbrev = game_data['teams']['away']['abbreviation']
    home_abbrev = game_data['teams']['home']['abbreviation']
    
    # access corresponding shift data and build dataframe
    shift_data_url = 'https://api.nhle.com/stats/rest/en/shiftcharts?cayenneExp=gameId='+str(code)
    req = Request(shift_data_url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    shift_json = json.loads(webpage.decode('utf-8')) ##webpage
    
    shift_player_id = []
    shift_player_name = []
    shift_player_team = []
    shift_period = []
    shift_start = []
    shift_end = []
    shifts = shift_json['data']
    
    for i in range(len(shifts)):
        shift_player_id.append(shifts[i]['playerId'])
        player_first = shifts[i]['firstName']
        player_last = shifts[i]['lastName']
        shift_player_name.append(player_first + ' ' + player_last)
        shift_player_team.append(shifts[i]['teamAbbrev'])
        shift_period.append(shifts[i]['period'])
        shift_start.append(shifts[i]['startTime'])
        shift_end.append(shifts[i]['endTime'])
        
    shift_df = pd.DataFrame({
        'playerID': shift_player_id,
        'player name': shift_player_name,
        'player team': shift_player_team,
        'period': shift_period,
        'shift start': shift_start,
        'shift end': shift_end
    })
    
    # iterate through all events and store as a row
    for i in range(len(events)):
    
        # add game_id to every row
        game_id.append(game_data['game']['pk'])

        # these keys exist for every event, whether hockey play or not
        event_result = events[i]['result']
        event_about = events[i]['about']
        
        eventIdx.append(event_about['eventIdx'])
        event_id.append(str(game_data['game']['pk']) + '-' + str(event_about['eventIdx'])) # primary key for nhl_events
        event.append(event_result['event'])
        eventCode.append(event_result['eventCode'])
        eventType.append(event_result['eventTypeId'])
        eventDescription.append(event_result['description'])
        period.append(event_about['period'])
        periodType.append(event_about['periodType'])
        periodTime.append(event_about['periodTime'])
        dateTime.append(event_about['dateTime'])
        awayGoals.append(event_about['goals']['away'])
        homeGoals.append(event_about['goals']['home'])
    
        # event is a hockey event (Shot, Goal, Hit, etc.)
        if 'players' in events[i]:

            player_info = events[i]['players']

            # event type is a Giveaway, Takeaway, or Missed Shot
            if len(player_info) == 1:
                player1_id.append(player_info[0]['player']['id'])
                player1_name.append(player_info[0]['player']['fullName'])
                player1_link.append(player_info[0]['player']['link'])
                player1_type.append(player_info[0]['playerType'])
                player2_id.append('NULL')
                player2_name.append('NULL')
                player2_link.append('NULL')
                player2_type.append('NULL')
                primaryAssister_id.append('NULL')
                primaryAssister_name.append('NULL')
                primaryAssister_link.append('NULL')
                primaryAssister_type.append('NULL')
                primaryAssister_total.append('NULL')
                secondaryAssister_id.append('NULL')
                secondaryAssister_name.append('NULL')
                secondaryAssister_link.append('NULL')
                secondaryAssister_type.append('NULL')
                secondaryAssister_total.append('NULL')
                penaltyServer_id.append('NULL')
                penaltyServer_name.append('NULL')
                penaltyServer_link.append('NULL')
                penaltyServer_type.append('NULL')
            # event type is not a Goal, Giveaway, Takeaway, or Missed Shot or it's a goal with no assists
            elif len(player_info) == 2:
                player1_id.append(player_info[0]['player']['id'])
                player1_name.append(player_info[0]['player']['fullName'])
                player1_link.append(player_info[0]['player']['link'])
                player1_type.append(player_info[0]['playerType'])
                player2_id.append(player_info[1]['player']['id'])
                player2_name.append(player_info[1]['player']['fullName'])
                player2_link.append(player_info[1]['player']['link'])
                player2_type.append(player_info[1]['playerType'])   
                primaryAssister_id.append('NULL')
                primaryAssister_name.append('NULL')
                primaryAssister_link.append('NULL')
                primaryAssister_type.append('NULL')
                primaryAssister_total.append('NULL')
                secondaryAssister_id.append('NULL')
                secondaryAssister_name.append('NULL')
                secondaryAssister_link.append('NULL')
                secondaryAssister_type.append('NULL')
                secondaryAssister_total.append('NULL')
                penaltyServer_id.append('NULL')
                penaltyServer_name.append('NULL')
                penaltyServer_link.append('NULL')
                penaltyServer_type.append('NULL')
            # event type is a Goal with one assist or a Penalty taken and served by different players
            elif len(player_info) == 3:
                player1_id.append(player_info[0]['player']['id'])
                player1_name.append(player_info[0]['player']['fullName'])
                player1_link.append(player_info[0]['player']['link'])
                player1_type.append(player_info[0]['playerType'])
                # event type is Goal
                if event_result['eventTypeId'] == 'GOAL':
                    primaryAssister_id.append(player_info[1]['player']['id'])
                    primaryAssister_name.append(player_info[1]['player']['fullName'])
                    primaryAssister_link.append(player_info[1]['player']['link'])
                    primaryAssister_type.append(player_info[1]['playerType'])
                    primaryAssister_total.append(player_info[1]['seasonTotal'])
                    player2_id.append(player_info[2]['player']['id'])
                    player2_name.append(player_info[2]['player']['fullName'])
                    player2_link.append(player_info[2]['player']['link'])
                    player2_type.append(player_info[2]['playerType'])
                    penaltyServer_id.append('NULL')
                    penaltyServer_name.append('NULL')
                    penaltyServer_link.append('NULL')
                    penaltyServer_type.append('NULL')
                # event type is Penalty
                elif event_result['eventTypeId'] == 'PENALTY':
                    player2_id.append(player_info[1]['player']['id'])
                    player2_name.append(player_info[1]['player']['fullName'])
                    player2_link.append(player_info[1]['player']['link'])
                    player2_type.append(player_info[1]['playerType'])
                    penaltyServer_id.append(player_info[2]['player']['id'])
                    penaltyServer_name.append(player_info[2]['player']['fullName'])
                    penaltyServer_link.append(player_info[2]['player']['link'])
                    penaltyServer_type.append(player_info[2]['playerType'])
                    primaryAssister_id.append('NULL')
                    primaryAssister_name.append('NULL')
                    primaryAssister_link.append('NULL')
                    primaryAssister_type.append('NULL')
                    primaryAssister_total.append('NULL')
                else:
                    print(game_data['game']['pk'])   
                secondaryAssister_id.append('NULL')
                secondaryAssister_name.append('NULL')
                secondaryAssister_link.append('NULL')
                secondaryAssister_type.append('NULL')
                secondaryAssister_total.append('NULL')
            # event type is a Goal with two assists
            elif len(player_info) == 4:
                player1_id.append(player_info[0]['player']['id'])
                player1_name.append(player_info[0]['player']['fullName'])
                player1_link.append(player_info[0]['player']['link'])
                player1_type.append(player_info[0]['playerType'])
                player2_id.append(player_info[3]['player']['id'])
                player2_name.append(player_info[3]['player']['fullName'])
                player2_link.append(player_info[3]['player']['link'])
                player2_type.append(player_info[3]['playerType']) 
                primaryAssister_id.append(player_info[1]['player']['id'])
                primaryAssister_name.append(player_info[1]['player']['fullName'])
                primaryAssister_link.append(player_info[1]['player']['link'])
                primaryAssister_type.append(player_info[1]['playerType'])
                primaryAssister_total.append(player_info[1]['seasonTotal'])
                secondaryAssister_id.append(player_info[2]['player']['id'])
                secondaryAssister_name.append(player_info[2]['player']['fullName'])
                secondaryAssister_link.append(player_info[2]['player']['link'])
                secondaryAssister_type.append(player_info[2]['playerType'])
                secondaryAssister_total.append(player_info[2]['seasonTotal'])
                penaltyServer_id.append('NULL')
                penaltyServer_name.append('NULL')
                penaltyServer_link.append('NULL')
                penaltyServer_type.append('NULL')

            # event is a Goal, Shot, or Penalty
            if 'secondaryType' in event_result:
                secondaryType.append(event_result['secondaryType'])
            else:
                secondaryType.append('NULL')

            # event is a Penalty
            if event_result['eventTypeId'] == 'PENALTY':
                penaltySeverity.append(event_result['penaltySeverity'])
                penaltyMinutes.append(event_result['penaltyMinutes'])
            else:
                penaltySeverity.append('NULL')
                penaltyMinutes.append('NULL')

            # event is a Goal
            if event_result['eventTypeId'] == 'GOAL':
                strength.append(event_result['strength']['code'])
                gwg.append(event_result['gameWinningGoal'])
                # not sure why but some goal events are missing emptyNet key
                if 'emptyNet' in event_result:
                    emptyNet.append(event_result['emptyNet'])
                else:
                    emptyNet.append('NULL')
                scorer_total.append(player_info[0]['seasonTotal'])
            else:
                strength.append('NULL')
                gwg.append('NULL')
                emptyNet.append('NULL')
                scorer_total.append('NULL')


            # regardless of event type
            team1_id.append(events[i]['team']['id'])
            team1_name.append(events[i]['team']['name'])
            team1_link.append(events[i]['team']['link'])
            team1_tricode.append(events[i]['team']['triCode'])
            # some events are missing coordinates, no idea why
            if events[i]['coordinates']:
                xCoord.append(events[i]['coordinates']['x'])
                yCoord.append(events[i]['coordinates']['y'])
            else:
                xCoord.append('Missing')
                yCoord.append('Missing')

            # players on ice for each event
            away_players_on_ice = []
            home_players_on_ice = []
            
            for shift in range(len(shift_df)):
                if event_about['period'] == shift_df['period'][shift]:
                    if event_during_shift(event_about['periodTime'], event_about['period'], shift_df['shift start'][shift],
                                            shift_df['shift end'][shift], shift_df['period'][shift]) == True:
                        if shift_df['player team'][shift] == away_abbrev:
                            away_players_on_ice.append(shift_df['player name'][shift])
                        elif shift_df['player team'][shift] == home_abbrev:
                            home_players_on_ice.append(shift_df['player name'][shift])

            if len(away_players_on_ice)>=1:
                away_p1.append(away_players_on_ice[0])
            else:
                away_p1.append('NULL')
            if len(away_players_on_ice)>=2:
                away_p2.append(away_players_on_ice[1])
            else:
                away_p2.append('NULL')
            if len(away_players_on_ice)>=3:
                away_p3.append(away_players_on_ice[2])
            else:
                away_p3.append('NULL')
            if len(away_players_on_ice)>=4:
                away_p4.append(away_players_on_ice[3])
            else:
                away_p4.append('NULL')
            if len(away_players_on_ice)>=5:
                away_p5.append(away_players_on_ice[4])
            else:
                away_p5.append('NULL')
            if len(away_players_on_ice)>=6:
                away_p6.append(away_players_on_ice[5])
            else:
                away_p6.append('NULL')

            if len(home_players_on_ice)>=1:
                home_p1.append(home_players_on_ice[0])
            else:
                home_p1.append('NULL')
            if len(home_players_on_ice)>=2:
                home_p2.append(home_players_on_ice[1])
            else:
                home_p2.append('NULL')
            if len(home_players_on_ice)>=3:
                home_p3.append(home_players_on_ice[2])
            else:
                home_p3.append('NULL')
            if len(home_players_on_ice)>=4:
                home_p4.append(home_players_on_ice[3])
            else:
                home_p4.append('NULL')
            if len(home_players_on_ice)>=5:
                home_p5.append(home_players_on_ice[4])
            else:
                home_p5.append('NULL')
            if len(home_players_on_ice)>=6:
                home_p6.append(home_players_on_ice[5])
            else:
                home_p6.append('NULL')


        # event is a non-hockey event (Game start/end, Period start/end, Stoppage, etc.)
        else:
            player1_id.append('NULL')
            player1_name.append('NULL')
            player1_link.append('NULL')
            player1_type.append('NULL')
            player2_id.append('NULL')
            player2_name.append('NULL')
            player2_link.append('NULL')
            player2_type.append('NULL')
            primaryAssister_id.append('NULL')
            primaryAssister_name.append('NULL')
            primaryAssister_link.append('NULL')
            primaryAssister_type.append('NULL')
            primaryAssister_total.append('NULL')
            secondaryAssister_id.append('NULL')
            secondaryAssister_name.append('NULL')
            secondaryAssister_link.append('NULL')
            secondaryAssister_type.append('NULL')
            secondaryAssister_total.append('NULL')
            secondaryType.append('N/A')
            penaltySeverity.append('N/A')
            penaltyMinutes.append('N/A')
            penaltyServer_id.append('NULL')
            penaltyServer_name.append('NULL')
            penaltyServer_link.append('NULL')
            penaltyServer_type.append('NULL')
            strength.append('NULL')
            gwg.append('NULL')
            emptyNet.append('NULL')
            scorer_total.append('NULL')
            xCoord.append('NULL')
            yCoord.append('NULL')
            team1_id.append('NULL')
            team1_name.append('NULL')
            team1_link.append('NULL')
            team1_tricode.append('NULL')
            away_p1.append('NULL')
            away_p2.append('NULL')
            away_p3.append('NULL')
            away_p4.append('NULL')
            away_p5.append('NULL')
            away_p6.append('NULL')
            home_p1.append('NULL')
            home_p2.append('NULL')
            home_p3.append('NULL')
            home_p4.append('NULL')
            home_p5.append('NULL')
            home_p6.append('NULL')

# convert resulting season-long lists into Dataframe
reg20192020_events_df = pd.DataFrame({
    'event_id': event_id,
    'game_id': game_id,
    'player1_id': player1_id,
    'player1_name': player1_name,
    'player1_link': player1_link,
    'player1_type': player1_type,
    'player2_id': player2_id,
    'player2_name': player2_name,
    'player2_link': player2_link,
    'player2_type': player2_type,
    'event': event,
    'eventCode': eventCode,
    'eventType': eventType,
    'eventDescription': eventDescription,
    'secondaryType': secondaryType,
    'xCoord': xCoord,
    'yCoord': yCoord,
    'primaryAssister_id': primaryAssister_id,
    'primaryAssister_name': primaryAssister_name,
    'primaryAssister_link': primaryAssister_link,
    'primaryAssister_type': primaryAssister_type,
    'primaryAssister_total': primaryAssister_total,
    'secondaryAssister_id': secondaryAssister_id,
    'secondaryAssister_name': secondaryAssister_name,
    'secondaryAssister_link': secondaryAssister_link,
    'secondaryAssister_type': secondaryAssister_type,
    'secondaryAssister_total': secondaryAssister_total,
    'penaltyServer_id': penaltyServer_id,
    'penaltyServer_name': penaltyServer_name,
    'penaltyServer_link': penaltyServer_link,
    'penaltyServer_type': penaltyServer_type,
    'strength': strength,
    'gwg': gwg,
    'emptyNet': emptyNet,
    'scorer_total': scorer_total,
    'penaltySeverity': penaltySeverity,
    'penaltyMinutes': penaltyMinutes,
    'eventIdx': eventIdx,
    'period': period,
    'periodType': periodType,
    'periodTime': periodTime,
    'dateTime': dateTime,
    'awayGoals': awayGoals,
    'homeGoals': homeGoals,
    'team1_id': team1_id,
    'team1_name': team1_name,
    'team1_link': team1_link,
    'team1_tricode': team1_tricode,
    'away p1': away_p1,
    'away p2': away_p2,
    'away p3': away_p3,
    'away p4': away_p4,
    'away p5': away_p5,
    'away p6': away_p6,
    'home p1': home_p1,
    'home p2': home_p2,
    'home p3': home_p3,
    'home p4': home_p4,
    'home p5': home_p5,
    'home p6': home_p6
})


In [ ]:
%store reg20192020_events_df

# 2019-2020 Playoffs

In [3]:
# create lists to store events for the entire season, to be converted to Dataframe
event_id = []
game_id = []
player1_id = []
player1_name = []
player1_link = []
player1_type = []
player2_id = []
player2_name = []
player2_link = []
player2_type = []
primaryAssister_id = []
primaryAssister_name = []
primaryAssister_link = []
primaryAssister_type = []
primaryAssister_total = []
secondaryAssister_id = []
secondaryAssister_name = []
secondaryAssister_link = []
secondaryAssister_type = []
secondaryAssister_total = []
penaltyServer_id = []
penaltyServer_name = []
penaltyServer_link = []
penaltyServer_type = []
strength = []
gwg = []
emptyNet = []
scorer_total = []
event = []
eventCode = []
eventType = []
eventDescription = []
secondaryType = []
penaltySeverity = []
penaltyMinutes = []
eventIdx = []
period = []
periodType = []
periodTime = []
dateTime = []
awayGoals = []
homeGoals = []
xCoord = []
yCoord = []
team1_id = []
team1_name = []
team1_link = []
team1_tricode = []
away_p1 = []
away_p2 = []
away_p3 = []
away_p4 = []
away_p5 = []
away_p6 = []
home_p1 = []
home_p2 = []
home_p3 = []
home_p4 = []
home_p5 = []
home_p6 = []

"""
For playoff games, the 2nd digit of the specific number gives the round of the playoffs, 
the 3rd digit specifies the matchup, and the 4th digit specifies the game (out of 7).
ex. Round 1, matchup 1, game 1 is 0111
8 matchups in 1st round: 011x, 012x, 013x, 014x, 015x, 016x, 017x, 018x where x depends on series length
4 matchups in 2nd round: 021x, 022x, 023x, 024x
2 matchups in 3rd round: 031x, 032x
1 matchup in 4th round: 041x
"""

root = 201903
game_codes = ['0111', '0112', '0113', '0114', '0115', '0116', '0117', '0121', '0122', '0123', '0124', '0125', '0126', '0127',
              '0131', '0132', '0133', '0134', '0135', '0136', '0137', '0141', '0142', '0143', '0144', '0145', '0146', '0147',
              '0151', '0152', '0153', '0154', '0155', '0156', '0157', '0161', '0162', '0163', '0164', '0165', '0166', '0167',
              '0171', '0172', '0173', '0174', '0175', '0176', '0177', '0181', '0182', '0183', '0184', '0185', '0186', '0187'
              '0211', '0212', '0213', '0214', '0215', '0216', '0217', '0221', '0222', '0223', '0224', '0225', '0226', '0227',
              '0231', '0232', '0233', '0234', '0235', '0236', '0237', '0241', '0242', '0243', '0244', '0245', '0246', '0247',
              '0311', '0312', '0313', '0314', '0315', '0316', '0317', '0321', '0322', '0323', '0324', '0325', '0326', '0327',
              '0411', '0412', '0413', '0414', '0415', '0416', '0417']

for game_code in game_codes:
    
    code = str(root)+game_code
    
    print(code)
    
    # access game data
    game_data_url = 'https://statsapi.web.nhl.com/api/v1/game/'+str(code)+'/feed/live'
    req = Request(game_data_url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    page_json = json.loads(webpage.decode('utf-8')) ##webpage
    game_data = page_json['gameData']

    # check if game has been played
    if game_data['status']['abstractGameState'] == 'Final':
        events = page_json['liveData']['plays']['allPlays']
    
        # get away and home team abbreviations
        away_abbrev = game_data['teams']['away']['abbreviation']
        home_abbrev = game_data['teams']['home']['abbreviation']

        # access corresponding shift data and build dataframe
        shift_data_url = 'https://api.nhle.com/stats/rest/en/shiftcharts?cayenneExp=gameId='+str(code)
        req = Request(shift_data_url, headers={'User-Agent': 'Mozilla/5.0'})
        webpage = urlopen(req).read()
        shift_json = json.loads(webpage.decode('utf-8')) ##webpage

        shift_player_id = []
        shift_player_name = []
        shift_player_team = []
        shift_period = []
        shift_start = []
        shift_end = []
        shifts = shift_json['data']

        for i in range(len(shifts)):
            shift_player_id.append(shifts[i]['playerId'])
            player_first = shifts[i]['firstName']
            player_last = shifts[i]['lastName']
            shift_player_name.append(player_first + ' ' + player_last)
            shift_player_team.append(shifts[i]['teamAbbrev'])
            shift_period.append(shifts[i]['period'])
            shift_start.append(shifts[i]['startTime'])
            shift_end.append(shifts[i]['endTime'])

        shift_df = pd.DataFrame({
            'playerID': shift_player_id,
            'player name': shift_player_name,
            'player team': shift_player_team,
            'period': shift_period,
            'shift start': shift_start,
            'shift end': shift_end
        })

        # iterate through all events and store as a row
        for i in range(len(events)):

            # add game_id to every row
            game_id.append(game_data['game']['pk'])

            # these keys exist for every event, whether hockey play or not
            event_result = events[i]['result']
            event_about = events[i]['about']

            eventIdx.append(event_about['eventIdx'])
            event_id.append(str(game_data['game']['pk']) + '-' + str(event_about['eventIdx'])) # primary key for nhl_events
            event.append(event_result['event'])
            eventCode.append(event_result['eventCode'])
            eventType.append(event_result['eventTypeId'])
            eventDescription.append(event_result['description'])
            period.append(event_about['period'])
            periodType.append(event_about['periodType'])
            periodTime.append(event_about['periodTime'])
            dateTime.append(event_about['dateTime'])
            awayGoals.append(event_about['goals']['away'])
            homeGoals.append(event_about['goals']['home'])

            # event is a hockey event (Shot, Goal, Hit, etc.)
            if 'players' in events[i]:

                player_info = events[i]['players']

                # event type is a Giveaway, Takeaway, or Missed Shot
                if len(player_info) == 1:
                    player1_id.append(player_info[0]['player']['id'])
                    player1_name.append(player_info[0]['player']['fullName'])
                    player1_link.append(player_info[0]['player']['link'])
                    player1_type.append(player_info[0]['playerType'])
                    player2_id.append('NULL')
                    player2_name.append('NULL')
                    player2_link.append('NULL')
                    player2_type.append('NULL')
                    primaryAssister_id.append('NULL')
                    primaryAssister_name.append('NULL')
                    primaryAssister_link.append('NULL')
                    primaryAssister_type.append('NULL')
                    primaryAssister_total.append('NULL')
                    secondaryAssister_id.append('NULL')
                    secondaryAssister_name.append('NULL')
                    secondaryAssister_link.append('NULL')
                    secondaryAssister_type.append('NULL')
                    secondaryAssister_total.append('NULL')
                    penaltyServer_id.append('NULL')
                    penaltyServer_name.append('NULL')
                    penaltyServer_link.append('NULL')
                    penaltyServer_type.append('NULL')
                # event type is not a Goal, Giveaway, Takeaway, or Missed Shot or it's a goal with no assists
                elif len(player_info) == 2:
                    player1_id.append(player_info[0]['player']['id'])
                    player1_name.append(player_info[0]['player']['fullName'])
                    player1_link.append(player_info[0]['player']['link'])
                    player1_type.append(player_info[0]['playerType'])
                    player2_id.append(player_info[1]['player']['id'])
                    player2_name.append(player_info[1]['player']['fullName'])
                    player2_link.append(player_info[1]['player']['link'])
                    player2_type.append(player_info[1]['playerType'])   
                    primaryAssister_id.append('NULL')
                    primaryAssister_name.append('NULL')
                    primaryAssister_link.append('NULL')
                    primaryAssister_type.append('NULL')
                    primaryAssister_total.append('NULL')
                    secondaryAssister_id.append('NULL')
                    secondaryAssister_name.append('NULL')
                    secondaryAssister_link.append('NULL')
                    secondaryAssister_type.append('NULL')
                    secondaryAssister_total.append('NULL')
                    penaltyServer_id.append('NULL')
                    penaltyServer_name.append('NULL')
                    penaltyServer_link.append('NULL')
                    penaltyServer_type.append('NULL')
                # event type is a Goal with one assist or a Penalty taken and served by different players
                elif len(player_info) == 3:
                    player1_id.append(player_info[0]['player']['id'])
                    player1_name.append(player_info[0]['player']['fullName'])
                    player1_link.append(player_info[0]['player']['link'])
                    player1_type.append(player_info[0]['playerType'])
                    # event type is Goal
                    if event_result['eventTypeId'] == 'GOAL':
                        primaryAssister_id.append(player_info[1]['player']['id'])
                        primaryAssister_name.append(player_info[1]['player']['fullName'])
                        primaryAssister_link.append(player_info[1]['player']['link'])
                        primaryAssister_type.append(player_info[1]['playerType'])
                        primaryAssister_total.append(player_info[1]['seasonTotal'])
                        player2_id.append(player_info[2]['player']['id'])
                        player2_name.append(player_info[2]['player']['fullName'])
                        player2_link.append(player_info[2]['player']['link'])
                        player2_type.append(player_info[2]['playerType'])
                        penaltyServer_id.append('NULL')
                        penaltyServer_name.append('NULL')
                        penaltyServer_link.append('NULL')
                        penaltyServer_type.append('NULL')
                    # event type is Penalty
                    elif event_result['eventTypeId'] == 'PENALTY':
                        player2_id.append(player_info[1]['player']['id'])
                        player2_name.append(player_info[1]['player']['fullName'])
                        player2_link.append(player_info[1]['player']['link'])
                        player2_type.append(player_info[1]['playerType'])
                        penaltyServer_id.append(player_info[2]['player']['id'])
                        penaltyServer_name.append(player_info[2]['player']['fullName'])
                        penaltyServer_link.append(player_info[2]['player']['link'])
                        penaltyServer_type.append(player_info[2]['playerType'])
                        primaryAssister_id.append('NULL')
                        primaryAssister_name.append('NULL')
                        primaryAssister_link.append('NULL')
                        primaryAssister_type.append('NULL')
                        primaryAssister_total.append('NULL')
                    # who fucking knows
                    else:
                        print(game_data['game']['pk'])   
                    secondaryAssister_id.append('NULL')
                    secondaryAssister_name.append('NULL')
                    secondaryAssister_link.append('NULL')
                    secondaryAssister_type.append('NULL')
                    secondaryAssister_total.append('NULL')
                # event type is a Goal with two assists
                elif len(player_info) == 4:
                    player1_id.append(player_info[0]['player']['id'])
                    player1_name.append(player_info[0]['player']['fullName'])
                    player1_link.append(player_info[0]['player']['link'])
                    player1_type.append(player_info[0]['playerType'])
                    player2_id.append(player_info[3]['player']['id'])
                    player2_name.append(player_info[3]['player']['fullName'])
                    player2_link.append(player_info[3]['player']['link'])
                    player2_type.append(player_info[3]['playerType']) 
                    primaryAssister_id.append(player_info[1]['player']['id'])
                    primaryAssister_name.append(player_info[1]['player']['fullName'])
                    primaryAssister_link.append(player_info[1]['player']['link'])
                    primaryAssister_type.append(player_info[1]['playerType'])
                    primaryAssister_total.append(player_info[1]['seasonTotal'])
                    secondaryAssister_id.append(player_info[2]['player']['id'])
                    secondaryAssister_name.append(player_info[2]['player']['fullName'])
                    secondaryAssister_link.append(player_info[2]['player']['link'])
                    secondaryAssister_type.append(player_info[2]['playerType'])
                    secondaryAssister_total.append(player_info[2]['seasonTotal'])
                    penaltyServer_id.append('NULL')
                    penaltyServer_name.append('NULL')
                    penaltyServer_link.append('NULL')
                    penaltyServer_type.append('NULL')

                # event is a Goal, Shot, or Penalty
                if 'secondaryType' in event_result:
                    secondaryType.append(event_result['secondaryType'])
                else:
                    secondaryType.append('NULL')

                # event is a Penalty
                if event_result['eventTypeId'] == 'PENALTY':
                    penaltySeverity.append(event_result['penaltySeverity'])
                    penaltyMinutes.append(event_result['penaltyMinutes'])
                else:
                    penaltySeverity.append('NULL')
                    penaltyMinutes.append('NULL')

                # event is a Goal
                if event_result['eventTypeId'] == 'GOAL':
                    strength.append(event_result['strength']['code'])
                    gwg.append(event_result['gameWinningGoal'])
                    # not sure why but some goal events are missing emptyNet key
                    if 'emptyNet' in event_result:
                        emptyNet.append(event_result['emptyNet'])
                    else:
                        emptyNet.append('NULL')
                    scorer_total.append(player_info[0]['seasonTotal'])
                else:
                    strength.append('NULL')
                    gwg.append('NULL')
                    emptyNet.append('NULL')
                    scorer_total.append('NULL')


                # regardless of event type
                team1_id.append(events[i]['team']['id'])
                team1_name.append(events[i]['team']['name'])
                team1_link.append(events[i]['team']['link'])
                team1_tricode.append(events[i]['team']['triCode'])
                # some events are missing coordinates, no idea why
                if events[i]['coordinates']:
                    xCoord.append(events[i]['coordinates']['x'])
                    yCoord.append(events[i]['coordinates']['y'])
                else:
                    xCoord.append('Missing')
                    yCoord.append('Missing')

                # players on ice for each event
                away_players_on_ice = []
                home_players_on_ice = []
                for shift in range(len(shift_df)):
                    if event_about['period'] == shift_df['period'][shift]:
                        if event_during_shift(event_about['periodTime'], event_about['period'], shift_df['shift start'][shift],
                                                shift_df['shift end'][shift], shift_df['period'][shift]) == True:
                            if shift_df['player team'][shift] == away_abbrev:
                                away_players_on_ice.append(shift_df['player name'][shift])
                            elif shift_df['player team'][shift] == home_abbrev:
                                home_players_on_ice.append(shift_df['player name'][shift])

                if len(away_players_on_ice)>=1:
                    away_p1.append(away_players_on_ice[0])
                else:
                    away_p1.append('NULL')
                if len(away_players_on_ice)>=2:
                    away_p2.append(away_players_on_ice[1])
                else:
                    away_p2.append('NULL')
                if len(away_players_on_ice)>=3:
                    away_p3.append(away_players_on_ice[2])
                else:
                    away_p3.append('NULL')
                if len(away_players_on_ice)>=4:
                    away_p4.append(away_players_on_ice[3])
                else:
                    away_p4.append('NULL')
                if len(away_players_on_ice)>=5:
                    away_p5.append(away_players_on_ice[4])
                else:
                    away_p5.append('NULL')
                if len(away_players_on_ice)>=6:
                    away_p6.append(away_players_on_ice[5])
                else:
                    away_p6.append('NULL')

                if len(home_players_on_ice)>=1:
                    home_p1.append(home_players_on_ice[0])
                else:
                    home_p1.append('NULL')
                if len(home_players_on_ice)>=2:
                    home_p2.append(home_players_on_ice[1])
                else:
                    home_p2.append('NULL')
                if len(home_players_on_ice)>=3:
                    home_p3.append(home_players_on_ice[2])
                else:
                    home_p3.append('NULL')
                if len(home_players_on_ice)>=4:
                    home_p4.append(home_players_on_ice[3])
                else:
                    home_p4.append('NULL')
                if len(home_players_on_ice)>=5:
                    home_p5.append(home_players_on_ice[4])
                else:
                    home_p5.append('NULL')
                if len(home_players_on_ice)>=6:
                    home_p6.append(home_players_on_ice[5])
                else:
                    home_p6.append('NULL')


            # event is a non-hockey event (Game start/end, Period start/end, Stoppage, etc.)
            else:
                player1_id.append('NULL')
                player1_name.append('NULL')
                player1_link.append('NULL')
                player1_type.append('NULL')
                player2_id.append('NULL')
                player2_name.append('NULL')
                player2_link.append('NULL')
                player2_type.append('NULL')
                primaryAssister_id.append('NULL')
                primaryAssister_name.append('NULL')
                primaryAssister_link.append('NULL')
                primaryAssister_type.append('NULL')
                primaryAssister_total.append('NULL')
                secondaryAssister_id.append('NULL')
                secondaryAssister_name.append('NULL')
                secondaryAssister_link.append('NULL')
                secondaryAssister_type.append('NULL')
                secondaryAssister_total.append('NULL')
                secondaryType.append('N/A')
                penaltySeverity.append('N/A')
                penaltyMinutes.append('N/A')
                penaltyServer_id.append('NULL')
                penaltyServer_name.append('NULL')
                penaltyServer_link.append('NULL')
                penaltyServer_type.append('NULL')
                strength.append('NULL')
                gwg.append('NULL')
                emptyNet.append('NULL')
                scorer_total.append('NULL')
                xCoord.append('NULL')
                yCoord.append('NULL')
                team1_id.append('NULL')
                team1_name.append('NULL')
                team1_link.append('NULL')
                team1_tricode.append('NULL')
                away_p1.append('NULL')
                away_p2.append('NULL')
                away_p3.append('NULL')
                away_p4.append('NULL')
                away_p5.append('NULL')
                away_p6.append('NULL')
                home_p1.append('NULL')
                home_p2.append('NULL')
                home_p3.append('NULL')
                home_p4.append('NULL')
                home_p5.append('NULL')
                home_p6.append('NULL')

# convert resulting season-long lists into Dataframe
playoff20192020_events_df = pd.DataFrame({
    'event_id': event_id,
    'game_id': game_id,
    'player1_id': player1_id,
    'player1_name': player1_name,
    'player1_link': player1_link,
    'player1_type': player1_type,
    'player2_id': player2_id,
    'player2_name': player2_name,
    'player2_link': player2_link,
    'player2_type': player2_type,
    'event': event,
    'eventCode': eventCode,
    'eventType': eventType,
    'eventDescription': eventDescription,
    'secondaryType': secondaryType,
    'xCoord': xCoord,
    'yCoord': yCoord,
    'primaryAssister_id': primaryAssister_id,
    'primaryAssister_name': primaryAssister_name,
    'primaryAssister_link': primaryAssister_link,
    'primaryAssister_type': primaryAssister_type,
    'primaryAssister_total': primaryAssister_total,
    'secondaryAssister_id': secondaryAssister_id,
    'secondaryAssister_name': secondaryAssister_name,
    'secondaryAssister_link': secondaryAssister_link,
    'secondaryAssister_type': secondaryAssister_type,
    'secondaryAssister_total': secondaryAssister_total,
    'penaltyServer_id': penaltyServer_id,
    'penaltyServer_name': penaltyServer_name,
    'penaltyServer_link': penaltyServer_link,
    'penaltyServer_type': penaltyServer_type,
    'strength': strength,
    'gwg': gwg,
    'emptyNet': emptyNet,
    'scorer_total': scorer_total,
    'penaltySeverity': penaltySeverity,
    'penaltyMinutes': penaltyMinutes,
    'eventIdx': eventIdx,
    'period': period,
    'periodType': periodType,
    'periodTime': periodTime,
    'dateTime': dateTime,
    'awayGoals': awayGoals,
    'homeGoals': homeGoals,
    'team1_id': team1_id,
    'team1_name': team1_name,
    'team1_link': team1_link,
    'team1_tricode': team1_tricode,
    'away p1': away_p1,
    'away p2': away_p2,
    'away p3': away_p3,
    'away p4': away_p4,
    'away p5': away_p5,
    'away p6': away_p6,
    'home p1': home_p1,
    'home p2': home_p2,
    'home p3': home_p3,
    'home p4': home_p4,
    'home p5': home_p5,
    'home p6': home_p6
})

2019030111
2019030112
2019030113
2019030114
2019030115
2019030116
2019030117
2019030121
2019030122
2019030123
2019030124
2019030125
2019030126
2019030127
2019030131
2019030132
2019030133
2019030134
2019030135
2019030136
2019030137
2019030141
2019030142
2019030143
2019030144
2019030145
2019030146
2019030147
2019030151
2019030152
2019030153
2019030154
2019030155
2019030156
2019030157
2019030161
2019030162
2019030163
2019030164
2019030165
2019030166
2019030167
2019030171
2019030172
2019030173
2019030174
2019030175
2019030176
2019030177
2019030181
2019030182
2019030183
2019030184
2019030185
2019030186


ConnectionAbortedError: [WinError 10053] An established connection was aborted by the software in your host machine

# 2020-2021 Regular Season

In [ ]:
# create lists to store events for the entire season, to be converted to Dataframe
event_id = []
game_id = []
player1_id = []
player1_name = []
player1_link = []
player1_type = []
player2_id = []
player2_name = []
player2_link = []
player2_type = []
primaryAssister_id = []
primaryAssister_name = []
primaryAssister_link = []
primaryAssister_type = []
primaryAssister_total = []
secondaryAssister_id = []
secondaryAssister_name = []
secondaryAssister_link = []
secondaryAssister_type = []
secondaryAssister_total = []
penaltyServer_id = []
penaltyServer_name = []
penaltyServer_link = []
penaltyServer_type = []
strength = []
gwg = []
emptyNet = []
scorer_total = []
event = []
eventCode = []
eventType = []
eventDescription = []
secondaryType = []
penaltySeverity = []
penaltyMinutes = []
eventIdx = []
period = []
periodType = []
periodTime = []
dateTime = []
awayGoals = []
homeGoals = []
xCoord = []
yCoord = []
team1_id = []
team1_name = []
team1_link = []
team1_tricode = []
away_p1 = []
away_p2 = []
away_p3 = []
away_p4 = []
away_p5 = []
away_p6 = []
home_p1 = []
home_p2 = []
home_p3 = []
home_p4 = []
home_p5 = []
home_p6 = []

# 2020-2021 reg. season only had 56 games per team and only 868 games total
root = '202002'

for game in range(1, 869): #1, 869
    
    if game<10:
        code = root+'000'+str(game)
    elif game<100:
        code = root+'00'+str(game)
    elif game<1000:
        code = root+'0'+str(game)
    else:
        code = root+str(game)
        
    print(code)
    
    # access game data
    game_data_url = 'https://statsapi.web.nhl.com/api/v1/game/'+str(code)+'/feed/live'
    req = Request(game_data_url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    page_json = json.loads(webpage.decode('utf-8')) ##webpage
    game_data = page_json['gameData']
    events = page_json['liveData']['plays']['allPlays']
    
    # get away and home team abbreviations
    away_abbrev = game_data['teams']['away']['abbreviation']
    home_abbrev = game_data['teams']['home']['abbreviation']
    
    # access corresponding shift data and build dataframe
    shift_data_url = 'https://api.nhle.com/stats/rest/en/shiftcharts?cayenneExp=gameId='+str(code)
    req = Request(shift_data_url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    shift_json = json.loads(webpage.decode('utf-8')) ##webpage
    
    shift_player_id = []
    shift_player_name = []
    shift_player_team = []
    shift_period = []
    shift_start = []
    shift_end = []
    shifts = shift_json['data']
    
    for i in range(len(shifts)):
        shift_player_id.append(shifts[i]['playerId'])
        player_first = shifts[i]['firstName']
        player_last = shifts[i]['lastName']
        shift_player_name.append(player_first + ' ' + player_last)
        shift_player_team.append(shifts[i]['teamAbbrev'])
        shift_period.append(shifts[i]['period'])
        shift_start.append(shifts[i]['startTime'])
        shift_end.append(shifts[i]['endTime'])
        
    shift_df = pd.DataFrame({
        'playerID': shift_player_id,
        'player name': shift_player_name,
        'player team': shift_player_team,
        'period': shift_period,
        'shift start': shift_start,
        'shift end': shift_end
    })
    
    # iterate through all events and store as a row
    for i in range(len(events)):
    
        # add game_id to every row
        game_id.append(game_data['game']['pk'])

        # these keys exist for every event, whether hockey play or not
        event_result = events[i]['result']
        event_about = events[i]['about']
        
        eventIdx.append(event_about['eventIdx'])
        event_id.append(str(game_data['game']['pk']) + '-' + str(event_about['eventIdx'])) # primary key for nhl_events
        event.append(event_result['event'])
        eventCode.append(event_result['eventCode'])
        eventType.append(event_result['eventTypeId'])
        eventDescription.append(event_result['description'])
        period.append(event_about['period'])
        periodType.append(event_about['periodType'])
        periodTime.append(event_about['periodTime'])
        dateTime.append(event_about['dateTime'])
        awayGoals.append(event_about['goals']['away'])
        homeGoals.append(event_about['goals']['home'])
    
        # event is a hockey event (Shot, Goal, Hit, etc.)
        if 'players' in events[i]:

            player_info = events[i]['players']

            # event type is a Giveaway, Takeaway, or Missed Shot
            if len(player_info) == 1:
                player1_id.append(player_info[0]['player']['id'])
                player1_name.append(player_info[0]['player']['fullName'])
                player1_link.append(player_info[0]['player']['link'])
                player1_type.append(player_info[0]['playerType'])
                player2_id.append('NULL')
                player2_name.append('NULL')
                player2_link.append('NULL')
                player2_type.append('NULL')
                primaryAssister_id.append('NULL')
                primaryAssister_name.append('NULL')
                primaryAssister_link.append('NULL')
                primaryAssister_type.append('NULL')
                primaryAssister_total.append('NULL')
                secondaryAssister_id.append('NULL')
                secondaryAssister_name.append('NULL')
                secondaryAssister_link.append('NULL')
                secondaryAssister_type.append('NULL')
                secondaryAssister_total.append('NULL')
                penaltyServer_id.append('NULL')
                penaltyServer_name.append('NULL')
                penaltyServer_link.append('NULL')
                penaltyServer_type.append('NULL')
            # event type is not a Goal, Giveaway, Takeaway, or Missed Shot or it's a goal with no assists
            elif len(player_info) == 2:
                player1_id.append(player_info[0]['player']['id'])
                player1_name.append(player_info[0]['player']['fullName'])
                player1_link.append(player_info[0]['player']['link'])
                player1_type.append(player_info[0]['playerType'])
                player2_id.append(player_info[1]['player']['id'])
                player2_name.append(player_info[1]['player']['fullName'])
                player2_link.append(player_info[1]['player']['link'])
                player2_type.append(player_info[1]['playerType'])   
                primaryAssister_id.append('NULL')
                primaryAssister_name.append('NULL')
                primaryAssister_link.append('NULL')
                primaryAssister_type.append('NULL')
                primaryAssister_total.append('NULL')
                secondaryAssister_id.append('NULL')
                secondaryAssister_name.append('NULL')
                secondaryAssister_link.append('NULL')
                secondaryAssister_type.append('NULL')
                secondaryAssister_total.append('NULL')
                penaltyServer_id.append('NULL')
                penaltyServer_name.append('NULL')
                penaltyServer_link.append('NULL')
                penaltyServer_type.append('NULL')
            # event type is a Goal with one assist or a Penalty taken and served by different players
            elif len(player_info) == 3:
                player1_id.append(player_info[0]['player']['id'])
                player1_name.append(player_info[0]['player']['fullName'])
                player1_link.append(player_info[0]['player']['link'])
                player1_type.append(player_info[0]['playerType'])
                # event type is Goal
                if event_result['eventTypeId'] == 'GOAL':
                    primaryAssister_id.append(player_info[1]['player']['id'])
                    primaryAssister_name.append(player_info[1]['player']['fullName'])
                    primaryAssister_link.append(player_info[1]['player']['link'])
                    primaryAssister_type.append(player_info[1]['playerType'])
                    primaryAssister_total.append(player_info[1]['seasonTotal'])
                    player2_id.append(player_info[2]['player']['id'])
                    player2_name.append(player_info[2]['player']['fullName'])
                    player2_link.append(player_info[2]['player']['link'])
                    player2_type.append(player_info[2]['playerType'])
                    penaltyServer_id.append('NULL')
                    penaltyServer_name.append('NULL')
                    penaltyServer_link.append('NULL')
                    penaltyServer_type.append('NULL')
                # event type is Penalty
                elif event_result['eventTypeId'] == 'PENALTY':
                    player2_id.append(player_info[1]['player']['id'])
                    player2_name.append(player_info[1]['player']['fullName'])
                    player2_link.append(player_info[1]['player']['link'])
                    player2_type.append(player_info[1]['playerType'])
                    penaltyServer_id.append(player_info[2]['player']['id'])
                    penaltyServer_name.append(player_info[2]['player']['fullName'])
                    penaltyServer_link.append(player_info[2]['player']['link'])
                    penaltyServer_type.append(player_info[2]['playerType'])
                    primaryAssister_id.append('NULL')
                    primaryAssister_name.append('NULL')
                    primaryAssister_link.append('NULL')
                    primaryAssister_type.append('NULL')
                    primaryAssister_total.append('NULL')
                # who fucking knows
                else:
                    print(game_data['game']['pk'])   
                secondaryAssister_id.append('NULL')
                secondaryAssister_name.append('NULL')
                secondaryAssister_link.append('NULL')
                secondaryAssister_type.append('NULL')
                secondaryAssister_total.append('NULL')
            # event type is a Goal with two assists
            elif len(player_info) == 4:
                player1_id.append(player_info[0]['player']['id'])
                player1_name.append(player_info[0]['player']['fullName'])
                player1_link.append(player_info[0]['player']['link'])
                player1_type.append(player_info[0]['playerType'])
                player2_id.append(player_info[3]['player']['id'])
                player2_name.append(player_info[3]['player']['fullName'])
                player2_link.append(player_info[3]['player']['link'])
                player2_type.append(player_info[3]['playerType']) 
                primaryAssister_id.append(player_info[1]['player']['id'])
                primaryAssister_name.append(player_info[1]['player']['fullName'])
                primaryAssister_link.append(player_info[1]['player']['link'])
                primaryAssister_type.append(player_info[1]['playerType'])
                primaryAssister_total.append(player_info[1]['seasonTotal'])
                secondaryAssister_id.append(player_info[2]['player']['id'])
                secondaryAssister_name.append(player_info[2]['player']['fullName'])
                secondaryAssister_link.append(player_info[2]['player']['link'])
                secondaryAssister_type.append(player_info[2]['playerType'])
                secondaryAssister_total.append(player_info[2]['seasonTotal'])
                penaltyServer_id.append('NULL')
                penaltyServer_name.append('NULL')
                penaltyServer_link.append('NULL')
                penaltyServer_type.append('NULL')
            else:
                print('What the fuck')

            # event is a Goal, Shot, or Penalty
            if 'secondaryType' in event_result:
                secondaryType.append(event_result['secondaryType'])
            else:
                secondaryType.append('NULL')

            # event is a Penalty
            if event_result['eventTypeId'] == 'PENALTY':
                penaltySeverity.append(event_result['penaltySeverity'])
                penaltyMinutes.append(event_result['penaltyMinutes'])
            else:
                penaltySeverity.append('NULL')
                penaltyMinutes.append('NULL')

            # event is a Goal
            if event_result['eventTypeId'] == 'GOAL':
                strength.append(event_result['strength']['code'])
                gwg.append(event_result['gameWinningGoal'])
                # not sure why but some goal events are missing emptyNet key
                if 'emptyNet' in event_result:
                    emptyNet.append(event_result['emptyNet'])
                else:
                    emptyNet.append('NULL')
                scorer_total.append(player_info[0]['seasonTotal'])
            else:
                strength.append('NULL')
                gwg.append('NULL')
                emptyNet.append('NULL')
                scorer_total.append('NULL')


            # regardless of event type
            team1_id.append(events[i]['team']['id'])
            team1_name.append(events[i]['team']['name'])
            team1_link.append(events[i]['team']['link'])
            team1_tricode.append(events[i]['team']['triCode'])
            # some events are missing coordinates, no idea why
            if events[i]['coordinates']:
                xCoord.append(events[i]['coordinates']['x'])
                yCoord.append(events[i]['coordinates']['y'])
            else:
                xCoord.append('Missing')
                yCoord.append('Missing')

            # players on ice for each event
            away_players_on_ice = []
            home_players_on_ice = []
            for shift in range(len(shift_df)):
                if event_about['period'] == shift_df['period'][shift]:
                    if event_during_shift(event_about['periodTime'], event_about['period'], shift_df['shift start'][shift],
                                            shift_df['shift end'][shift], shift_df['period'][shift]) == True:
                        if shift_df['player team'][shift] == away_abbrev:
                            away_players_on_ice.append(shift_df['player name'][shift])
                        elif shift_df['player team'][shift] == home_abbrev:
                            home_players_on_ice.append(shift_df['player name'][shift])

            if len(away_players_on_ice)>=1:
                away_p1.append(away_players_on_ice[0])
            else:
                away_p1.append('NULL')
            if len(away_players_on_ice)>=2:
                away_p2.append(away_players_on_ice[1])
            else:
                away_p2.append('NULL')
            if len(away_players_on_ice)>=3:
                away_p3.append(away_players_on_ice[2])
            else:
                away_p3.append('NULL')
            if len(away_players_on_ice)>=4:
                away_p4.append(away_players_on_ice[3])
            else:
                away_p4.append('NULL')
            if len(away_players_on_ice)>=5:
                away_p5.append(away_players_on_ice[4])
            else:
                away_p5.append('NULL')
            if len(away_players_on_ice)>=6:
                away_p6.append(away_players_on_ice[5])
            else:
                away_p6.append('NULL')

            if len(home_players_on_ice)>=1:
                home_p1.append(home_players_on_ice[0])
            else:
                home_p1.append('NULL')
            if len(home_players_on_ice)>=2:
                home_p2.append(home_players_on_ice[1])
            else:
                home_p2.append('NULL')
            if len(home_players_on_ice)>=3:
                home_p3.append(home_players_on_ice[2])
            else:
                home_p3.append('NULL')
            if len(home_players_on_ice)>=4:
                home_p4.append(home_players_on_ice[3])
            else:
                home_p4.append('NULL')
            if len(home_players_on_ice)>=5:
                home_p5.append(home_players_on_ice[4])
            else:
                home_p5.append('NULL')
            if len(home_players_on_ice)>=6:
                home_p6.append(home_players_on_ice[5])
            else:
                home_p6.append('NULL')


        # event is a non-hockey event (Game start/end, Period start/end, Stoppage, etc.)
        else:
            player1_id.append('NULL')
            player1_name.append('NULL')
            player1_link.append('NULL')
            player1_type.append('NULL')
            player2_id.append('NULL')
            player2_name.append('NULL')
            player2_link.append('NULL')
            player2_type.append('NULL')
            primaryAssister_id.append('NULL')
            primaryAssister_name.append('NULL')
            primaryAssister_link.append('NULL')
            primaryAssister_type.append('NULL')
            primaryAssister_total.append('NULL')
            secondaryAssister_id.append('NULL')
            secondaryAssister_name.append('NULL')
            secondaryAssister_link.append('NULL')
            secondaryAssister_type.append('NULL')
            secondaryAssister_total.append('NULL')
            secondaryType.append('N/A')
            penaltySeverity.append('N/A')
            penaltyMinutes.append('N/A')
            penaltyServer_id.append('NULL')
            penaltyServer_name.append('NULL')
            penaltyServer_link.append('NULL')
            penaltyServer_type.append('NULL')
            strength.append('NULL')
            gwg.append('NULL')
            emptyNet.append('NULL')
            scorer_total.append('NULL')
            xCoord.append('NULL')
            yCoord.append('NULL')
            team1_id.append('NULL')
            team1_name.append('NULL')
            team1_link.append('NULL')
            team1_tricode.append('NULL')
            away_p1.append('NULL')
            away_p2.append('NULL')
            away_p3.append('NULL')
            away_p4.append('NULL')
            away_p5.append('NULL')
            away_p6.append('NULL')
            home_p1.append('NULL')
            home_p2.append('NULL')
            home_p3.append('NULL')
            home_p4.append('NULL')
            home_p5.append('NULL')
            home_p6.append('NULL')

# convert resulting season-long lists into Dataframe
reg20202021_events_df = pd.DataFrame({
    'event_id': event_id,
    'game_id': game_id,
    'player1_id': player1_id,
    'player1_name': player1_name,
    'player1_link': player1_link,
    'player1_type': player1_type,
    'player2_id': player2_id,
    'player2_name': player2_name,
    'player2_link': player2_link,
    'player2_type': player2_type,
    'event': event,
    'eventCode': eventCode,
    'eventType': eventType,
    'eventDescription': eventDescription,
    'secondaryType': secondaryType,
    'xCoord': xCoord,
    'yCoord': yCoord,
    'primaryAssister_id': primaryAssister_id,
    'primaryAssister_name': primaryAssister_name,
    'primaryAssister_link': primaryAssister_link,
    'primaryAssister_type': primaryAssister_type,
    'primaryAssister_total': primaryAssister_total,
    'secondaryAssister_id': secondaryAssister_id,
    'secondaryAssister_name': secondaryAssister_name,
    'secondaryAssister_link': secondaryAssister_link,
    'secondaryAssister_type': secondaryAssister_type,
    'secondaryAssister_total': secondaryAssister_total,
    'penaltyServer_id': penaltyServer_id,
    'penaltyServer_name': penaltyServer_name,
    'penaltyServer_link': penaltyServer_link,
    'penaltyServer_type': penaltyServer_type,
    'strength': strength,
    'gwg': gwg,
    'emptyNet': emptyNet,
    'scorer_total': scorer_total,
    'penaltySeverity': penaltySeverity,
    'penaltyMinutes': penaltyMinutes,
    'eventIdx': eventIdx,
    'period': period,
    'periodType': periodType,
    'periodTime': periodTime,
    'dateTime': dateTime,
    'awayGoals': awayGoals,
    'homeGoals': homeGoals,
    'team1_id': team1_id,
    'team1_name': team1_name,
    'team1_link': team1_link,
    'team1_tricode': team1_tricode,
    'away p1': away_p1,
    'away p2': away_p2,
    'away p3': away_p3,
    'away p4': away_p4,
    'away p5': away_p5,
    'away p6': away_p6,
    'home p1': home_p1,
    'home p2': home_p2,
    'home p3': home_p3,
    'home p4': home_p4,
    'home p5': home_p5,
    'home p6': home_p6
})

In [ ]:
%store reg20202021_events_df

# 2020-2021 Playoffs

In [ ]:
# create lists to store events for the entire season, to be converted to Dataframe
event_id = []
game_id = []
player1_id = []
player1_name = []
player1_link = []
player1_type = []
player2_id = []
player2_name = []
player2_link = []
player2_type = []
primaryAssister_id = []
primaryAssister_name = []
primaryAssister_link = []
primaryAssister_type = []
primaryAssister_total = []
secondaryAssister_id = []
secondaryAssister_name = []
secondaryAssister_link = []
secondaryAssister_type = []
secondaryAssister_total = []
penaltyServer_id = []
penaltyServer_name = []
penaltyServer_link = []
penaltyServer_type = []
strength = []
gwg = []
emptyNet = []
scorer_total = []
event = []
eventCode = []
eventType = []
eventDescription = []
secondaryType = []
penaltySeverity = []
penaltyMinutes = []
eventIdx = []
period = []
periodType = []
periodTime = []
dateTime = []
awayGoals = []
homeGoals = []
xCoord = []
yCoord = []
team1_id = []
team1_name = []
team1_link = []
team1_tricode = []
away_p1 = []
away_p2 = []
away_p3 = []
away_p4 = []
away_p5 = []
away_p6 = []
home_p1 = []
home_p2 = []
home_p3 = []
home_p4 = []
home_p5 = []
home_p6 = []

"""
For playoff games, the 2nd digit of the specific number gives the round of the playoffs, 
the 3rd digit specifies the matchup, and the 4th digit specifies the game (out of 7).
ex. Round 1, matchup 1, game 1 is 0111
8 matchups in 1st round: 011x, 012x, 013x, 014x, 015x, 016x, 017x, 018x where x depends on series length
4 matchups in 2nd round: 021x, 022x, 023x, 024x
2 matchups in 3rd round: 031x, 032x
1 matchup in 4th round: 041x

Series lengths:
1st round - 6, 5, 6, 6, 4, 7, 7, 4
2nd round - 6, 5, 6, 4
3rd round - 6, 7
4th round - 5
"""

root = 202003
game_codes = ['0111', '0112', '0113', '0114', '0115', '0116', '0121', '0122', '0123', '0124', '0125',
              '0131', '0132', '0133', '0134', '0135', '0136', '0141', '0142', '0143', '0144', '0145',
              '0146', '0151', '0152', '0153', '0154', '0161', '0162', '0163', '0164', '0165', '0166',
              '0167', '0171', '0172', '0173', '0174', '0175', '0176', '0177', '0181', '0182', '0183', 
              '0184', '0211', '0212', '0213', '0214', '0215', '0216', '0221', '0222', '0223', '0224', 
              '0225', '0231', '0232', '0233', '0234', '0235', '0236', '0241', '0242', '0243', '0244', 
              '0311', '0312', '0313', '0314', '0315', '0316', '0321', '0322', '0323', '0324', '0325',
              '0326', '0327', '0411', '0412', '0413', '0414', '0415']

for game_code in game_codes:
    
    code = str(root)+game_code
    
    print(code)
    
    # access game data
    game_data_url = 'https://statsapi.web.nhl.com/api/v1/game/'+str(code)+'/feed/live'
    req = Request(game_data_url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    page_json = json.loads(webpage.decode('utf-8')) ##webpage
    game_data = page_json['gameData']
    events = page_json['liveData']['plays']['allPlays']
    
    # get away and home team abbreviations
    away_abbrev = game_data['teams']['away']['abbreviation']
    home_abbrev = game_data['teams']['home']['abbreviation']
    
    # access corresponding shift data and build dataframe
    shift_data_url = 'https://api.nhle.com/stats/rest/en/shiftcharts?cayenneExp=gameId='+str(code)
    req = Request(shift_data_url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    shift_json = json.loads(webpage.decode('utf-8')) ##webpage
    
    shift_player_id = []
    shift_player_name = []
    shift_player_team = []
    shift_period = []
    shift_start = []
    shift_end = []
    shifts = shift_json['data']
    
    for i in range(len(shifts)):
        shift_player_id.append(shifts[i]['playerId'])
        player_first = shifts[i]['firstName']
        player_last = shifts[i]['lastName']
        shift_player_name.append(player_first + ' ' + player_last)
        shift_player_team.append(shifts[i]['teamAbbrev'])
        shift_period.append(shifts[i]['period'])
        shift_start.append(shifts[i]['startTime'])
        shift_end.append(shifts[i]['endTime'])
        
    shift_df = pd.DataFrame({
        'playerID': shift_player_id,
        'player name': shift_player_name,
        'player team': shift_player_team,
        'period': shift_period,
        'shift start': shift_start,
        'shift end': shift_end
    })
    
    # iterate through all events and store as a row
    for i in range(len(events)):
    
        # add game_id to every row
        game_id.append(game_data['game']['pk'])

        # these keys exist for every event, whether hockey play or not
        event_result = events[i]['result']
        event_about = events[i]['about']
        
        eventIdx.append(event_about['eventIdx'])
        event_id.append(str(game_data['game']['pk']) + '-' + str(event_about['eventIdx'])) # primary key for nhl_events
        event.append(event_result['event'])
        eventCode.append(event_result['eventCode'])
        eventType.append(event_result['eventTypeId'])
        eventDescription.append(event_result['description'])
        period.append(event_about['period'])
        periodType.append(event_about['periodType'])
        periodTime.append(event_about['periodTime'])
        dateTime.append(event_about['dateTime'])
        awayGoals.append(event_about['goals']['away'])
        homeGoals.append(event_about['goals']['home'])
    
        # event is a hockey event (Shot, Goal, Hit, etc.)
        if 'players' in events[i]:

            player_info = events[i]['players']

            # event type is a Giveaway, Takeaway, or Missed Shot
            if len(player_info) == 1:
                player1_id.append(player_info[0]['player']['id'])
                player1_name.append(player_info[0]['player']['fullName'])
                player1_link.append(player_info[0]['player']['link'])
                player1_type.append(player_info[0]['playerType'])
                player2_id.append('NULL')
                player2_name.append('NULL')
                player2_link.append('NULL')
                player2_type.append('NULL')
                primaryAssister_id.append('NULL')
                primaryAssister_name.append('NULL')
                primaryAssister_link.append('NULL')
                primaryAssister_type.append('NULL')
                primaryAssister_total.append('NULL')
                secondaryAssister_id.append('NULL')
                secondaryAssister_name.append('NULL')
                secondaryAssister_link.append('NULL')
                secondaryAssister_type.append('NULL')
                secondaryAssister_total.append('NULL')
                penaltyServer_id.append('NULL')
                penaltyServer_name.append('NULL')
                penaltyServer_link.append('NULL')
                penaltyServer_type.append('NULL')
            # event type is not a Goal, Giveaway, Takeaway, or Missed Shot or it's a goal with no assists
            elif len(player_info) == 2:
                player1_id.append(player_info[0]['player']['id'])
                player1_name.append(player_info[0]['player']['fullName'])
                player1_link.append(player_info[0]['player']['link'])
                player1_type.append(player_info[0]['playerType'])
                player2_id.append(player_info[1]['player']['id'])
                player2_name.append(player_info[1]['player']['fullName'])
                player2_link.append(player_info[1]['player']['link'])
                player2_type.append(player_info[1]['playerType'])   
                primaryAssister_id.append('NULL')
                primaryAssister_name.append('NULL')
                primaryAssister_link.append('NULL')
                primaryAssister_type.append('NULL')
                primaryAssister_total.append('NULL')
                secondaryAssister_id.append('NULL')
                secondaryAssister_name.append('NULL')
                secondaryAssister_link.append('NULL')
                secondaryAssister_type.append('NULL')
                secondaryAssister_total.append('NULL')
                penaltyServer_id.append('NULL')
                penaltyServer_name.append('NULL')
                penaltyServer_link.append('NULL')
                penaltyServer_type.append('NULL')
            # event type is a Goal with one assist or a Penalty taken and served by different players
            elif len(player_info) == 3:
                player1_id.append(player_info[0]['player']['id'])
                player1_name.append(player_info[0]['player']['fullName'])
                player1_link.append(player_info[0]['player']['link'])
                player1_type.append(player_info[0]['playerType'])
                # event type is Goal
                if event_result['eventTypeId'] == 'GOAL':
                    primaryAssister_id.append(player_info[1]['player']['id'])
                    primaryAssister_name.append(player_info[1]['player']['fullName'])
                    primaryAssister_link.append(player_info[1]['player']['link'])
                    primaryAssister_type.append(player_info[1]['playerType'])
                    primaryAssister_total.append(player_info[1]['seasonTotal'])
                    player2_id.append(player_info[2]['player']['id'])
                    player2_name.append(player_info[2]['player']['fullName'])
                    player2_link.append(player_info[2]['player']['link'])
                    player2_type.append(player_info[2]['playerType'])
                    penaltyServer_id.append('NULL')
                    penaltyServer_name.append('NULL')
                    penaltyServer_link.append('NULL')
                    penaltyServer_type.append('NULL')
                # event type is Penalty
                elif event_result['eventTypeId'] == 'PENALTY':
                    player2_id.append(player_info[1]['player']['id'])
                    player2_name.append(player_info[1]['player']['fullName'])
                    player2_link.append(player_info[1]['player']['link'])
                    player2_type.append(player_info[1]['playerType'])
                    penaltyServer_id.append(player_info[2]['player']['id'])
                    penaltyServer_name.append(player_info[2]['player']['fullName'])
                    penaltyServer_link.append(player_info[2]['player']['link'])
                    penaltyServer_type.append(player_info[2]['playerType'])
                    primaryAssister_id.append('NULL')
                    primaryAssister_name.append('NULL')
                    primaryAssister_link.append('NULL')
                    primaryAssister_type.append('NULL')
                    primaryAssister_total.append('NULL')
                else:
                    print(game_data['game']['pk'])   
                secondaryAssister_id.append('NULL')
                secondaryAssister_name.append('NULL')
                secondaryAssister_link.append('NULL')
                secondaryAssister_type.append('NULL')
                secondaryAssister_total.append('NULL')
            # event type is a Goal with two assists
            elif len(player_info) == 4:
                player1_id.append(player_info[0]['player']['id'])
                player1_name.append(player_info[0]['player']['fullName'])
                player1_link.append(player_info[0]['player']['link'])
                player1_type.append(player_info[0]['playerType'])
                player2_id.append(player_info[3]['player']['id'])
                player2_name.append(player_info[3]['player']['fullName'])
                player2_link.append(player_info[3]['player']['link'])
                player2_type.append(player_info[3]['playerType']) 
                primaryAssister_id.append(player_info[1]['player']['id'])
                primaryAssister_name.append(player_info[1]['player']['fullName'])
                primaryAssister_link.append(player_info[1]['player']['link'])
                primaryAssister_type.append(player_info[1]['playerType'])
                primaryAssister_total.append(player_info[1]['seasonTotal'])
                secondaryAssister_id.append(player_info[2]['player']['id'])
                secondaryAssister_name.append(player_info[2]['player']['fullName'])
                secondaryAssister_link.append(player_info[2]['player']['link'])
                secondaryAssister_type.append(player_info[2]['playerType'])
                secondaryAssister_total.append(player_info[2]['seasonTotal'])
                penaltyServer_id.append('NULL')
                penaltyServer_name.append('NULL')
                penaltyServer_link.append('NULL')
                penaltyServer_type.append('NULL')

            # event is a Goal, Shot, or Penalty
            if 'secondaryType' in event_result:
                secondaryType.append(event_result['secondaryType'])
            else:
                secondaryType.append('NULL')

            # event is a Penalty
            if event_result['eventTypeId'] == 'PENALTY':
                penaltySeverity.append(event_result['penaltySeverity'])
                penaltyMinutes.append(event_result['penaltyMinutes'])
            else:
                penaltySeverity.append('NULL')
                penaltyMinutes.append('NULL')

            # event is a Goal
            if event_result['eventTypeId'] == 'GOAL':
                strength.append(event_result['strength']['code'])
                gwg.append(event_result['gameWinningGoal'])
                # not sure why but some goal events are missing emptyNet key
                if 'emptyNet' in event_result:
                    emptyNet.append(event_result['emptyNet'])
                else:
                    emptyNet.append('NULL')
                scorer_total.append(player_info[0]['seasonTotal'])
            else:
                strength.append('NULL')
                gwg.append('NULL')
                emptyNet.append('NULL')
                scorer_total.append('NULL')


            # regardless of event type
            team1_id.append(events[i]['team']['id'])
            team1_name.append(events[i]['team']['name'])
            team1_link.append(events[i]['team']['link'])
            team1_tricode.append(events[i]['team']['triCode'])
            # some events are missing coordinates, no idea why
            if events[i]['coordinates']:
                xCoord.append(events[i]['coordinates']['x'])
                yCoord.append(events[i]['coordinates']['y'])
            else:
                xCoord.append('Missing')
                yCoord.append('Missing')

            # players on ice for each event
            away_players_on_ice = []
            home_players_on_ice = []
            for shift in range(len(shift_df)):
                if event_about['period'] == shift_df['period'][shift]:
                    if event_during_shift(event_about['periodTime'], event_about['period'], shift_df['shift start'][shift],
                                            shift_df['shift end'][shift], shift_df['period'][shift]) == True:
                        if shift_df['player team'][shift] == away_abbrev:
                            away_players_on_ice.append(shift_df['player name'][shift])
                        elif shift_df['player team'][shift] == home_abbrev:
                            home_players_on_ice.append(shift_df['player name'][shift])

            if len(away_players_on_ice)>=1:
                away_p1.append(away_players_on_ice[0])
            else:
                away_p1.append('NULL')
            if len(away_players_on_ice)>=2:
                away_p2.append(away_players_on_ice[1])
            else:
                away_p2.append('NULL')
            if len(away_players_on_ice)>=3:
                away_p3.append(away_players_on_ice[2])
            else:
                away_p3.append('NULL')
            if len(away_players_on_ice)>=4:
                away_p4.append(away_players_on_ice[3])
            else:
                away_p4.append('NULL')
            if len(away_players_on_ice)>=5:
                away_p5.append(away_players_on_ice[4])
            else:
                away_p5.append('NULL')
            if len(away_players_on_ice)>=6:
                away_p6.append(away_players_on_ice[5])
            else:
                away_p6.append('NULL')

            if len(home_players_on_ice)>=1:
                home_p1.append(home_players_on_ice[0])
            else:
                home_p1.append('NULL')
            if len(home_players_on_ice)>=2:
                home_p2.append(home_players_on_ice[1])
            else:
                home_p2.append('NULL')
            if len(home_players_on_ice)>=3:
                home_p3.append(home_players_on_ice[2])
            else:
                home_p3.append('NULL')
            if len(home_players_on_ice)>=4:
                home_p4.append(home_players_on_ice[3])
            else:
                home_p4.append('NULL')
            if len(home_players_on_ice)>=5:
                home_p5.append(home_players_on_ice[4])
            else:
                home_p5.append('NULL')
            if len(home_players_on_ice)>=6:
                home_p6.append(home_players_on_ice[5])
            else:
                home_p6.append('NULL')


        # event is a non-hockey event (Game start/end, Period start/end, Stoppage, etc.)
        else:
            player1_id.append('NULL')
            player1_name.append('NULL')
            player1_link.append('NULL')
            player1_type.append('NULL')
            player2_id.append('NULL')
            player2_name.append('NULL')
            player2_link.append('NULL')
            player2_type.append('NULL')
            primaryAssister_id.append('NULL')
            primaryAssister_name.append('NULL')
            primaryAssister_link.append('NULL')
            primaryAssister_type.append('NULL')
            primaryAssister_total.append('NULL')
            secondaryAssister_id.append('NULL')
            secondaryAssister_name.append('NULL')
            secondaryAssister_link.append('NULL')
            secondaryAssister_type.append('NULL')
            secondaryAssister_total.append('NULL')
            secondaryType.append('N/A')
            penaltySeverity.append('N/A')
            penaltyMinutes.append('N/A')
            penaltyServer_id.append('NULL')
            penaltyServer_name.append('NULL')
            penaltyServer_link.append('NULL')
            penaltyServer_type.append('NULL')
            strength.append('NULL')
            gwg.append('NULL')
            emptyNet.append('NULL')
            scorer_total.append('NULL')
            xCoord.append('NULL')
            yCoord.append('NULL')
            team1_id.append('NULL')
            team1_name.append('NULL')
            team1_link.append('NULL')
            team1_tricode.append('NULL')
            away_p1.append('NULL')
            away_p2.append('NULL')
            away_p3.append('NULL')
            away_p4.append('NULL')
            away_p5.append('NULL')
            away_p6.append('NULL')
            home_p1.append('NULL')
            home_p2.append('NULL')
            home_p3.append('NULL')
            home_p4.append('NULL')
            home_p5.append('NULL')
            home_p6.append('NULL')

# convert resulting season-long lists into Dataframe
playoff20202021_events_df = pd.DataFrame({
    'event_id': event_id,
    'game_id': game_id,
    'player1_id': player1_id,
    'player1_name': player1_name,
    'player1_link': player1_link,
    'player1_type': player1_type,
    'player2_id': player2_id,
    'player2_name': player2_name,
    'player2_link': player2_link,
    'player2_type': player2_type,
    'event': event,
    'eventCode': eventCode,
    'eventType': eventType,
    'eventDescription': eventDescription,
    'secondaryType': secondaryType,
    'xCoord': xCoord,
    'yCoord': yCoord,
    'primaryAssister_id': primaryAssister_id,
    'primaryAssister_name': primaryAssister_name,
    'primaryAssister_link': primaryAssister_link,
    'primaryAssister_type': primaryAssister_type,
    'primaryAssister_total': primaryAssister_total,
    'secondaryAssister_id': secondaryAssister_id,
    'secondaryAssister_name': secondaryAssister_name,
    'secondaryAssister_link': secondaryAssister_link,
    'secondaryAssister_type': secondaryAssister_type,
    'secondaryAssister_total': secondaryAssister_total,
    'penaltyServer_id': penaltyServer_id,
    'penaltyServer_name': penaltyServer_name,
    'penaltyServer_link': penaltyServer_link,
    'penaltyServer_type': penaltyServer_type,
    'strength': strength,
    'gwg': gwg,
    'emptyNet': emptyNet,
    'scorer_total': scorer_total,
    'penaltySeverity': penaltySeverity,
    'penaltyMinutes': penaltyMinutes,
    'eventIdx': eventIdx,
    'period': period,
    'periodType': periodType,
    'periodTime': periodTime,
    'dateTime': dateTime,
    'awayGoals': awayGoals,
    'homeGoals': homeGoals,
    'team1_id': team1_id,
    'team1_name': team1_name,
    'team1_link': team1_link,
    'team1_tricode': team1_tricode,
    'away p1': away_p1,
    'away p2': away_p2,
    'away p3': away_p3,
    'away p4': away_p4,
    'away p5': away_p5,
    'away p6': away_p6,
    'home p1': home_p1,
    'home p2': home_p2,
    'home p3': home_p3,
    'home p4': home_p4,
    'home p5': home_p5,
    'home p6': home_p6
})

# 2021-2022 Regular Season

In [8]:
# create lists to store events for the entire season, to be converted to Dataframe
event_id = []
game_id = []
player1_id = []
player1_name = []
player1_link = []
player1_type = []
player2_id = []
player2_name = []
player2_link = []
player2_type = []
primaryAssister_id = []
primaryAssister_name = []
primaryAssister_link = []
primaryAssister_type = []
primaryAssister_total = []
secondaryAssister_id = []
secondaryAssister_name = []
secondaryAssister_link = []
secondaryAssister_type = []
secondaryAssister_total = []
penaltyServer_id = []
penaltyServer_name = []
penaltyServer_link = []
penaltyServer_type = []
strength = []
gwg = []
emptyNet = []
scorer_total = []
event = []
eventCode = []
eventType = []
eventDescription = []
secondaryType = []
penaltySeverity = []
penaltyMinutes = []
eventIdx = []
period = []
periodType = []
periodTime = []
dateTime = []
awayGoals = []
homeGoals = []
xCoord = []
yCoord = []
team1_id = []
team1_name = []
team1_link = []
team1_tricode = []
away_p1 = []
away_p2 = []
away_p3 = []
away_p4 = []
away_p5 = []
away_p6 = []
home_p1 = []
home_p2 = []
home_p3 = []
home_p4 = []
home_p5 = []
home_p6 = []

# 2021-2022 regular season will have 1312 games total
root = '202102'

for game in range(1, 1313):
    
    if game<10:
        code = root+'000'+str(game)
    elif game<100:
        code = root+'00'+str(game)
    elif game<1000:
        code = root+'0'+str(game)
    else:
        code = root+str(game)
        
#     print(code)
    
    # access game data page and pull relevant JSON
    game_data_url = 'https://statsapi.web.nhl.com/api/v1/game/'+str(code)+'/feed/live'
    req = Request(game_data_url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    page_json = json.loads(webpage)
    game_data = page_json['gameData']

    # check if game has been played
    if game_data['status']['abstractGameState'] == 'Final':
        events = page_json['liveData']['plays']['allPlays']

        # get away and home team abbreviations
        away_abbrev = game_data['teams']['away']['abbreviation']
        home_abbrev = game_data['teams']['home']['abbreviation']

        # access corresponding shift data and build dataframe
        shift_data_url = 'https://api.nhle.com/stats/rest/en/shiftcharts?cayenneExp=gameId='+str(code)
        req = Request(shift_data_url, headers={'User-Agent': 'Mozilla/5.0'})
        webpage = urlopen(req).read()
        shift_json = json.loads(webpage.decode('utf-8'))

        shift_player_id = []
        shift_player_name = []
        shift_player_team = []
        shift_period = []
        shift_start = []
        shift_end = []
        shifts = shift_json['data']

        for i in range(len(shifts)):
            shift_player_id.append(shifts[i]['playerId'])
            player_first = shifts[i]['firstName']
            player_last = shifts[i]['lastName']
            shift_player_name.append(player_first + ' ' + player_last)
            shift_player_team.append(shifts[i]['teamAbbrev'])
            shift_period.append(shifts[i]['period'])
            shift_start.append(shifts[i]['startTime'])
            shift_end.append(shifts[i]['endTime'])

        shift_df = pd.DataFrame({
            'playerID': shift_player_id,
            'player name': shift_player_name,
            'player team': shift_player_team,
            'period': shift_period,
            'shift start': shift_start,
            'shift end': shift_end
        })

        # iterate through all events and store as a row
        for i in range(len(events)):

            # add game_id to every row
            game_id.append(game_data['game']['pk'])

            # these keys exist for every event, whether hockey play or not
            event_result = events[i]['result']
            event_about = events[i]['about']

            eventIdx.append(event_about['eventIdx'])
            event_id.append(str(game_data['game']['pk']) + '-' + str(event_about['eventIdx'])) # primary key for nhl_events
            event.append(event_result['event'])
            eventCode.append(event_result['eventCode'])
            eventType.append(event_result['eventTypeId'])
            eventDescription.append(event_result['description'])
            period.append(event_about['period'])
            periodType.append(event_about['periodType'])
            periodTime.append(event_about['periodTime'])
            dateTime.append(event_about['dateTime'])
            awayGoals.append(event_about['goals']['away'])
            homeGoals.append(event_about['goals']['home'])

            # event is a hockey event (Shot, Goal, Hit, etc.)
            if 'players' in events[i]:

                player_info = events[i]['players']

                # event type is a Giveaway, Takeaway, or Missed Shot
                if len(player_info) == 1:
                    player1_id.append(player_info[0]['player']['id'])
                    player1_name.append(player_info[0]['player']['fullName'])
                    player1_link.append(player_info[0]['player']['link'])
                    player1_type.append(player_info[0]['playerType'])
                    player2_id.append('NULL')
                    player2_name.append('NULL')
                    player2_link.append('NULL')
                    player2_type.append('NULL')
                    primaryAssister_id.append('NULL')
                    primaryAssister_name.append('NULL')
                    primaryAssister_link.append('NULL')
                    primaryAssister_type.append('NULL')
                    primaryAssister_total.append('NULL')
                    secondaryAssister_id.append('NULL')
                    secondaryAssister_name.append('NULL')
                    secondaryAssister_link.append('NULL')
                    secondaryAssister_type.append('NULL')
                    secondaryAssister_total.append('NULL')
                    penaltyServer_id.append('NULL')
                    penaltyServer_name.append('NULL')
                    penaltyServer_link.append('NULL')
                    penaltyServer_type.append('NULL')
                # event type is not a Goal, Giveaway, Takeaway, or Missed Shot or it's a goal with no assists
                elif len(player_info) == 2:
                    player1_id.append(player_info[0]['player']['id'])
                    player1_name.append(player_info[0]['player']['fullName'])
                    player1_link.append(player_info[0]['player']['link'])
                    player1_type.append(player_info[0]['playerType'])
                    player2_id.append(player_info[1]['player']['id'])
                    player2_name.append(player_info[1]['player']['fullName'])
                    player2_link.append(player_info[1]['player']['link'])
                    player2_type.append(player_info[1]['playerType'])   
                    primaryAssister_id.append('NULL')
                    primaryAssister_name.append('NULL')
                    primaryAssister_link.append('NULL')
                    primaryAssister_type.append('NULL')
                    primaryAssister_total.append('NULL')
                    secondaryAssister_id.append('NULL')
                    secondaryAssister_name.append('NULL')
                    secondaryAssister_link.append('NULL')
                    secondaryAssister_type.append('NULL')
                    secondaryAssister_total.append('NULL')
                    penaltyServer_id.append('NULL')
                    penaltyServer_name.append('NULL')
                    penaltyServer_link.append('NULL')
                    penaltyServer_type.append('NULL')
                # event type is a Goal with one assist or a Penalty taken and served by different players
                elif len(player_info) == 3:
                    player1_id.append(player_info[0]['player']['id'])
                    player1_name.append(player_info[0]['player']['fullName'])
                    player1_link.append(player_info[0]['player']['link'])
                    player1_type.append(player_info[0]['playerType'])
                    # event type is Goal
                    if event_result['eventTypeId'] == 'GOAL':
                        primaryAssister_id.append(player_info[1]['player']['id'])
                        primaryAssister_name.append(player_info[1]['player']['fullName'])
                        primaryAssister_link.append(player_info[1]['player']['link'])
                        primaryAssister_type.append(player_info[1]['playerType'])
                        primaryAssister_total.append(player_info[1]['seasonTotal'])
                        player2_id.append(player_info[2]['player']['id'])
                        player2_name.append(player_info[2]['player']['fullName'])
                        player2_link.append(player_info[2]['player']['link'])
                        player2_type.append(player_info[2]['playerType'])
                        penaltyServer_id.append('NULL')
                        penaltyServer_name.append('NULL')
                        penaltyServer_link.append('NULL')
                        penaltyServer_type.append('NULL')
                    # event type is Penalty
                    elif event_result['eventTypeId'] == 'PENALTY':
                        player2_id.append(player_info[1]['player']['id'])
                        player2_name.append(player_info[1]['player']['fullName'])
                        player2_link.append(player_info[1]['player']['link'])
                        player2_type.append(player_info[1]['playerType'])
                        penaltyServer_id.append(player_info[2]['player']['id'])
                        penaltyServer_name.append(player_info[2]['player']['fullName'])
                        penaltyServer_link.append(player_info[2]['player']['link'])
                        penaltyServer_type.append(player_info[2]['playerType'])
                        primaryAssister_id.append('NULL')
                        primaryAssister_name.append('NULL')
                        primaryAssister_link.append('NULL')
                        primaryAssister_type.append('NULL')
                        primaryAssister_total.append('NULL')
                    else:
                        print(game_data['game']['pk'])   
                    secondaryAssister_id.append('NULL')
                    secondaryAssister_name.append('NULL')
                    secondaryAssister_link.append('NULL')
                    secondaryAssister_type.append('NULL')
                    secondaryAssister_total.append('NULL')
                # event type is a Goal with two assists
                elif len(player_info) == 4:
                    player1_id.append(player_info[0]['player']['id'])
                    player1_name.append(player_info[0]['player']['fullName'])
                    player1_link.append(player_info[0]['player']['link'])
                    player1_type.append(player_info[0]['playerType'])
                    player2_id.append(player_info[3]['player']['id'])
                    player2_name.append(player_info[3]['player']['fullName'])
                    player2_link.append(player_info[3]['player']['link'])
                    player2_type.append(player_info[3]['playerType']) 
                    primaryAssister_id.append(player_info[1]['player']['id'])
                    primaryAssister_name.append(player_info[1]['player']['fullName'])
                    primaryAssister_link.append(player_info[1]['player']['link'])
                    primaryAssister_type.append(player_info[1]['playerType'])
                    primaryAssister_total.append(player_info[1]['seasonTotal'])
                    secondaryAssister_id.append(player_info[2]['player']['id'])
                    secondaryAssister_name.append(player_info[2]['player']['fullName'])
                    secondaryAssister_link.append(player_info[2]['player']['link'])
                    secondaryAssister_type.append(player_info[2]['playerType'])
                    secondaryAssister_total.append(player_info[2]['seasonTotal'])
                    penaltyServer_id.append('NULL')
                    penaltyServer_name.append('NULL')
                    penaltyServer_link.append('NULL')
                    penaltyServer_type.append('NULL')
                else:
                    print('What the fuck')

                # event is a Goal, Shot, or Penalty
                if 'secondaryType' in event_result:
                    secondaryType.append(event_result['secondaryType'])
                else:
                    secondaryType.append('NULL')

                # event is a Penalty
                if event_result['eventTypeId'] == 'PENALTY':
                    penaltySeverity.append(event_result['penaltySeverity'])
                    penaltyMinutes.append(event_result['penaltyMinutes'])
                else:
                    penaltySeverity.append('NULL')
                    penaltyMinutes.append('NULL')

                # event is a Goal
                if event_result['eventTypeId'] == 'GOAL':
                    strength.append(event_result['strength']['code'])
                    gwg.append(event_result['gameWinningGoal'])
                    # not sure why but some goal events are missing emptyNet key
                    if 'emptyNet' in event_result:
                        emptyNet.append(event_result['emptyNet'])
                    else:
                        emptyNet.append('NULL')
                    scorer_total.append(player_info[0]['seasonTotal'])
                else:
                    strength.append('NULL')
                    gwg.append('NULL')
                    emptyNet.append('NULL')
                    scorer_total.append('NULL')


                # regardless of event type
                team1_id.append(events[i]['team']['id'])
                team1_name.append(events[i]['team']['name'])
                team1_link.append(events[i]['team']['link'])
                team1_tricode.append(events[i]['team']['triCode'])
                # some events are missing coordinates, no idea why
                if events[i]['coordinates']:
                    xCoord.append(events[i]['coordinates']['x'])
                    yCoord.append(events[i]['coordinates']['y'])
                else:
                    xCoord.append('Missing')
                    yCoord.append('Missing')

                # players on ice for each event
                away_players_on_ice = []
                home_players_on_ice = []
                for shift in range(len(shift_df)):
                    if event_about['period'] == shift_df['period'][shift]:
                        if event_during_shift(event_about['periodTime'], event_about['period'], shift_df['shift start'][shift],
                                                shift_df['shift end'][shift], shift_df['period'][shift]) == True:
                            if shift_df['player team'][shift] == away_abbrev:
                                away_players_on_ice.append(shift_df['player name'][shift])
                            elif shift_df['player team'][shift] == home_abbrev:
                                home_players_on_ice.append(shift_df['player name'][shift])

                if len(away_players_on_ice)>=1:
                    away_p1.append(away_players_on_ice[0])
                else:
                    away_p1.append('NULL')
                if len(away_players_on_ice)>=2:
                    away_p2.append(away_players_on_ice[1])
                else:
                    away_p2.append('NULL')
                if len(away_players_on_ice)>=3:
                    away_p3.append(away_players_on_ice[2])
                else:
                    away_p3.append('NULL')
                if len(away_players_on_ice)>=4:
                    away_p4.append(away_players_on_ice[3])
                else:
                    away_p4.append('NULL')
                if len(away_players_on_ice)>=5:
                    away_p5.append(away_players_on_ice[4])
                else:
                    away_p5.append('NULL')
                if len(away_players_on_ice)>=6:
                    away_p6.append(away_players_on_ice[5])
                else:
                    away_p6.append('NULL')

                if len(home_players_on_ice)>=1:
                    home_p1.append(home_players_on_ice[0])
                else:
                    home_p1.append('NULL')
                if len(home_players_on_ice)>=2:
                    home_p2.append(home_players_on_ice[1])
                else:
                    home_p2.append('NULL')
                if len(home_players_on_ice)>=3:
                    home_p3.append(home_players_on_ice[2])
                else:
                    home_p3.append('NULL')
                if len(home_players_on_ice)>=4:
                    home_p4.append(home_players_on_ice[3])
                else:
                    home_p4.append('NULL')
                if len(home_players_on_ice)>=5:
                    home_p5.append(home_players_on_ice[4])
                else:
                    home_p5.append('NULL')
                if len(home_players_on_ice)>=6:
                    home_p6.append(home_players_on_ice[5])
                else:
                    home_p6.append('NULL')


            # event is a non-hockey event (Game start/end, Period start/end, Stoppage, etc.)
            else:
                player1_id.append('NULL')
                player1_name.append('NULL')
                player1_link.append('NULL')
                player1_type.append('NULL')
                player2_id.append('NULL')
                player2_name.append('NULL')
                player2_link.append('NULL')
                player2_type.append('NULL')
                primaryAssister_id.append('NULL')
                primaryAssister_name.append('NULL')
                primaryAssister_link.append('NULL')
                primaryAssister_type.append('NULL')
                primaryAssister_total.append('NULL')
                secondaryAssister_id.append('NULL')
                secondaryAssister_name.append('NULL')
                secondaryAssister_link.append('NULL')
                secondaryAssister_type.append('NULL')
                secondaryAssister_total.append('NULL')
                secondaryType.append('N/A')
                penaltySeverity.append('N/A')
                penaltyMinutes.append('N/A')
                penaltyServer_id.append('NULL')
                penaltyServer_name.append('NULL')
                penaltyServer_link.append('NULL')
                penaltyServer_type.append('NULL')
                strength.append('NULL')
                gwg.append('NULL')
                emptyNet.append('NULL')
                scorer_total.append('NULL')
                xCoord.append('NULL')
                yCoord.append('NULL')
                team1_id.append('NULL')
                team1_name.append('NULL')
                team1_link.append('NULL')
                team1_tricode.append('NULL')
                away_p1.append('NULL')
                away_p2.append('NULL')
                away_p3.append('NULL')
                away_p4.append('NULL')
                away_p5.append('NULL')
                away_p6.append('NULL')
                home_p1.append('NULL')
                home_p2.append('NULL')
                home_p3.append('NULL')
                home_p4.append('NULL')
                home_p5.append('NULL')
                home_p6.append('NULL')

    else:
        event_id.append('Not played')
        game_id.append(game_data['game']['pk'])
        player1_id.append('Not played')
        player1_name.append('Not played')
        player1_link.append('Not played')
        player1_type.append('Not played')
        player2_id.append('Not played')
        player2_name.append('Not played')
        player2_link.append('Not played')
        player2_type.append('Not played')
        primaryAssister_id.append('Not played')
        primaryAssister_name.append('Not played')
        primaryAssister_link.append('Not played')
        primaryAssister_type.append('Not played')
        primaryAssister_total.append('Not played')
        secondaryAssister_id.append('Not played')
        secondaryAssister_name.append('Not played')
        secondaryAssister_link.append('Not played')
        secondaryAssister_type.append('Not played')
        secondaryAssister_total.append('Not played')
        penaltyServer_id.append('Not played')
        penaltyServer_name.append('Not played')
        penaltyServer_link.append('Not played')
        penaltyServer_type.append('Not played')
        strength.append('Not played')
        gwg.append('Not played')
        emptyNet.append('Not played')
        scorer_total.append('Not played')
        event.append('Not played')
        eventCode.append('Not played')
        eventType.append('Not played')
        eventDescription.append('Not played')
        secondaryType.append('Not played')
        penaltySeverity.append('Not played')
        penaltyMinutes.append('Not played')
        eventIdx.append('Not played')
        period.append('Not played')
        periodType.append('Not played')
        periodTime.append('Not played')
        dateTime.append('Not played')
        awayGoals.append('Not played')
        homeGoals.append('Not played')
        xCoord.append('Not played')
        yCoord.append('Not played')
        team1_id.append('Not played')
        team1_name.append('Not played')
        team1_link.append('Not played')
        team1_tricode.append('Not played')
        away_p1.append('Not played')
        away_p2.append('Not played')
        away_p3.append('Not played')
        away_p4.append('Not played')
        away_p5.append('Not played')
        away_p6.append('Not played')
        home_p1.append('Not played')
        home_p2.append('Not played')
        home_p3.append('Not played')
        home_p4.append('Not played')
        home_p5.append('Not played')
        home_p6.append('Not played')

# convert resulting season-long lists into Dataframe
reg20212022_events_df = pd.DataFrame({
    'event_id': event_id,
    'game_id': game_id,
    'player1_id': player1_id,
    'player1_name': player1_name,
    'player1_link': player1_link,
    'player1_type': player1_type,
    'player2_id': player2_id,
    'player2_name': player2_name,
    'player2_link': player2_link,
    'player2_type': player2_type,
    'event': event,
    'eventCode': eventCode,
    'eventType': eventType,
    'eventDescription': eventDescription,
    'secondaryType': secondaryType,
    'xCoord': xCoord,
    'yCoord': yCoord,
    'primaryAssister_id': primaryAssister_id,
    'primaryAssister_name': primaryAssister_name,
    'primaryAssister_link': primaryAssister_link,
    'primaryAssister_type': primaryAssister_type,
    'primaryAssister_total': primaryAssister_total,
    'secondaryAssister_id': secondaryAssister_id,
    'secondaryAssister_name': secondaryAssister_name,
    'secondaryAssister_link': secondaryAssister_link,
    'secondaryAssister_type': secondaryAssister_type,
    'secondaryAssister_total': secondaryAssister_total,
    'penaltyServer_id': penaltyServer_id,
    'penaltyServer_name': penaltyServer_name,
    'penaltyServer_link': penaltyServer_link,
    'penaltyServer_type': penaltyServer_type,
    'strength': strength,
    'gwg': gwg,
    'emptyNet': emptyNet,
    'scorer_total': scorer_total,
    'penaltySeverity': penaltySeverity,
    'penaltyMinutes': penaltyMinutes,
    'eventIdx': eventIdx,
    'period': period,
    'periodType': periodType,
    'periodTime': periodTime,
    'dateTime': dateTime,
    'awayGoals': awayGoals,
    'homeGoals': homeGoals,
    'team1_id': team1_id,
    'team1_name': team1_name,
    'team1_link': team1_link,
    'team1_tricode': team1_tricode,
    'away p1': away_p1,
    'away p2': away_p2,
    'away p3': away_p3,
    'away p4': away_p4,
    'away p5': away_p5,
    'away p6': away_p6,
    'home p1': home_p1,
    'home p2': home_p2,
    'home p3': home_p3,
    'home p4': home_p4,
    'home p5': home_p5,
    'home p6': home_p6
})

2021020620
2021020621
2021020622
2021020623
2021020624
2021020625
2021020626
2021020627
2021020628
2021020629
2021020630
2021020631
2021020632
2021020633
2021020634
2021020635
2021020636
2021020637
2021020638
2021020639


# Function Form

In [1]:
# TODO, in progress on nhl_api_scraper notebook